BENV0093 TBPS6 - Wind Speed Data Processing

In [1]:
import numpy as np
import xarray as xr
import rasterio
from rasterio.transform import from_origin
import pickle
from pathlib import Path

print("="*80)
print("WIND SPEED DATA PROCESSING")
print("="*80)

# Configuration
BASE_DIR = Path('D:/BENV0093')
DATASET_DIR = BASE_DIR / 'Dateset_T2'
OUTPUT_DIR = BASE_DIR / 'outputs'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

WIND_SPEED_NC = DATASET_DIR / 'WindSpeed' / 'wind_speed.nc'
UK_BOUNDARY_SHP = BASE_DIR / 'UK_ITL1_Boundaries.shp'
WIND_100M_TIF = OUTPUT_DIR / 'wind_speed_100m_original.tif'
WIND_SCORE_TIF = OUTPUT_DIR / 'wind_suitability_score_original.tif'
REFERENCE_GRID_PKL = OUTPUT_DIR / 'reference_grid.pkl'

DATA_HEIGHT = 10
TURBINE_HEIGHT = 100
HELLMAN_EXPONENT = 0.143
WIND_EXTRAPOLATION_FACTOR = (TURBINE_HEIGHT / DATA_HEIGHT) ** HELLMAN_EXPONENT

WIND_THRESHOLD_MIN = 6.0
WIND_THRESHOLD_MAX = 9.0
MAX_SCORE = 10.0
SCORE_SLOPE = MAX_SCORE / (WIND_THRESHOLD_MAX - WIND_THRESHOLD_MIN)

NODATA_VALUE = -9999
CRS_EPSG = 27700

print(f"Extrapolation factor: {WIND_EXTRAPOLATION_FACTOR:.4f}")

if not WIND_SPEED_NC.exists():
    print(f"ERROR: Wind speed file not found: {WIND_SPEED_NC}")
    exit(1)

# Step 1: Load data
print("\nStep 1: Loading wind speed data")
ds = xr.open_dataset(WIND_SPEED_NC)
print(ds)

wind_var_names = ['wind_speed', 'ws', 'u10', 'wind', 'speed', 'Band1']
wind_10m = None
for var_name in wind_var_names:
    if var_name in ds.data_vars or var_name in ds:
        wind_10m = ds[var_name]
        print(f"Found variable: '{var_name}'")
        break

if wind_10m is None:
    print(f"ERROR: Wind speed variable not found")
    print(f"Available: {list(ds.data_vars.keys())}")
    exit(1)

if wind_10m.ndim == 3:
    # Try to find the time/season dimension
    possible_time_dims = ['time', 'season', 't']
    time_dim = None
    for dim in possible_time_dims:
        if dim in wind_10m.dims:
            time_dim = dim
            break
    
    if time_dim:
        wind_10m_array = wind_10m.mean(dim=time_dim).values
        print(f"Averaged over '{time_dim}' dimension")
    else:
        print(f"ERROR: Could not find time/season dimension")
        print(f"Available dimensions: {wind_10m.dims}")
        exit(1)
elif wind_10m.ndim == 2:
    wind_10m_array = wind_10m.values
else:
    print(f"ERROR: Unexpected dimensions: {wind_10m.ndim}")
    exit(1)

if 'x' in ds.coords and 'y' in ds.coords:
    x_coords = ds['x'].values
    y_coords = ds['y'].values
elif 'lon' in ds.coords and 'lat' in ds.coords:
    x_coords = ds['lon'].values
    y_coords = ds['lat'].values
else:
    print("ERROR: Coordinates not found")
    exit(1)

print(f"Shape: {wind_10m_array.shape}")
print(f"Range: {np.nanmin(wind_10m_array):.2f} - {np.nanmax(wind_10m_array):.2f} m/s")
print(f"Mean: {np.nanmean(wind_10m_array):.2f} m/s")

# Step 2: Extrapolate to 100m
print(f"\nStep 2: Extrapolating to {TURBINE_HEIGHT}m")
wind_100m_array = wind_10m_array * WIND_EXTRAPOLATION_FACTOR
print(f"Formula: U_100 = U_10 * {WIND_EXTRAPOLATION_FACTOR:.4f}")
print(f"Range: {np.nanmin(wind_100m_array):.2f} - {np.nanmax(wind_100m_array):.2f} m/s")
print(f"Mean: {np.nanmean(wind_100m_array):.2f} m/s")

# Step 3: Calculate suitability scores
print("\nStep 3: Calculating suitability scores")
wind_score_array = np.full_like(wind_100m_array, np.nan, dtype=np.float32)
valid_mask = ~np.isnan(wind_100m_array)

mask_low = valid_mask & (wind_100m_array < WIND_THRESHOLD_MIN)
wind_score_array[mask_low] = 0.0

mask_medium = valid_mask & (wind_100m_array >= WIND_THRESHOLD_MIN) & (wind_100m_array <= WIND_THRESHOLD_MAX)
wind_score_array[mask_medium] = SCORE_SLOPE * (wind_100m_array[mask_medium] - WIND_THRESHOLD_MIN)

mask_high = valid_mask & (wind_100m_array > WIND_THRESHOLD_MAX)
wind_score_array[mask_high] = MAX_SCORE

print(f"v < {WIND_THRESHOLD_MIN}: {mask_low.sum():,} cells")
print(f"{WIND_THRESHOLD_MIN} <= v <= {WIND_THRESHOLD_MAX}: {mask_medium.sum():,} cells")
print(f"v > {WIND_THRESHOLD_MAX}: {mask_high.sum():,} cells")

valid_scores = wind_score_array[~np.isnan(wind_score_array)]
print(f"Score mean: {valid_scores.mean():.2f}")

# Step 4: Setup GeoTIFF parameters
print("\nStep 4: Setting up GeoTIFF parameters")
rows, cols = wind_100m_array.shape
x_spacing = abs(x_coords[1] - x_coords[0]) if len(x_coords) > 1 else 1000
y_spacing = abs(y_coords[1] - y_coords[0]) if len(y_coords) > 1 else 1000

# Check if y coordinates are increasing or decreasing
if y_coords[1] > y_coords[0]:
    # Y increases (bottom to top) - need to flip data
    wind_100m_array = np.flipud(wind_100m_array)
    wind_score_array = np.flipud(wind_score_array)
    print("Flipped data vertically (y coordinates increasing)")

x_min = x_coords.min() - x_spacing / 2
y_max = y_coords.max() + y_spacing / 2
transform = from_origin(x_min, y_max, x_spacing, y_spacing)
extent = [x_min, x_min + cols * x_spacing, y_max - rows * y_spacing, y_max]
print(f"Pixel size: {x_spacing:.2f} x {y_spacing:.2f} m")

# Step 5: Save GeoTIFF files
print("\nStep 5: Saving GeoTIFF files")
wind_100m_save = np.where(np.isnan(wind_100m_array), NODATA_VALUE, wind_100m_array)
wind_score_save = np.where(np.isnan(wind_score_array), NODATA_VALUE, wind_score_array)

metadata = {
    'driver': 'GTiff',
    'dtype': 'float32',
    'nodata': NODATA_VALUE,
    'width': cols,
    'height': rows,
    'count': 1,
    'crs': f'EPSG:{CRS_EPSG}',
    'transform': transform,
    'compress': 'lzw'
}

with rasterio.open(WIND_100M_TIF, 'w', **metadata) as dst:
    dst.write(wind_100m_save.astype('float32'), 1)
print(f"Saved: {WIND_100M_TIF}")

with rasterio.open(WIND_SCORE_TIF, 'w', **metadata) as dst:
    dst.write(wind_score_save.astype('float32'), 1)
print(f"Saved: {WIND_SCORE_TIF}")

# Step 6: Save reference grid
print("\nStep 6: Saving reference grid")
reference_data = {
    'extent': extent,
    'transform': transform,
    'rows': rows,
    'cols': cols,
    'shape': (rows, cols),
    'pixel_size': (x_spacing, y_spacing),
    'crs': CRS_EPSG,
    'x_spacing': x_spacing,
    'y_spacing': y_spacing,
    'x_min': x_min,
    'y_max': y_max,
}

with open(REFERENCE_GRID_PKL, 'wb') as f:
    pickle.dump(reference_data, f)
print(f"Saved: {REFERENCE_GRID_PKL}")

print("\n" + "="*80)
print("PROCESSING COMPLETE")
print("="*80)
print(f"Resolution: {rows} x {cols}")
print(f"Valid data: {(~np.isnan(wind_100m_array)).sum():,} cells")

ds.close()

WIND SPEED DATA PROCESSING
Extrapolation factor: 1.3900

Step 1: Loading wind speed data
<xarray.Dataset> Size: 987kB
Dimensions:      (y: 235, x: 131, season: 4)
Coordinates:
  * y            (y) int32 940B 12500 17500 22500 ... 1172500 1177500 1182500
  * x            (x) int32 524B 2500 7500 12500 17500 ... 642500 647500 652500
  * season       (season) object 32B 'spring' 'summer' 'autumn' 'winter'
Data variables:
    wind_speed   (season, y, x) float64 985kB ...
    spatial_ref  int32 4B ...
Found variable: 'wind_speed'
Averaged over 'season' dimension
Shape: (235, 131)
Range: 2.73 - 7.50 m/s
Mean: 4.26 m/s

Step 2: Extrapolating to 100m
Formula: U_100 = U_10 * 1.3900
Range: 3.80 - 10.42 m/s
Mean: 5.92 m/s

Step 3: Calculating suitability scores
v < 6.0: 6,689 cells
6.0 <= v <= 9.0: 3,636 cells
v > 9.0: 72 cells
Score mean: 1.13

Step 4: Setting up GeoTIFF parameters
Flipped data vertically (y coordinates increasing)
Pixel size: 5000.00 x 5000.00 m

Step 5: Saving GeoTIFF files
Sa

Wind Speed Visualization

In [2]:
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, BoundaryNorm
import matplotlib.patches as mpatches
import geopandas as gpd
import pickle
from pathlib import Path

print("="*80)
print("WIND SPEED VISUALIZATION")
print("="*80)

# Configuration
BASE_DIR = Path('D:/BENV0093')
OUTPUT_DIR = BASE_DIR / 'outputs'
MAPS_DIR = OUTPUT_DIR / 'maps'
MAPS_DIR.mkdir(parents=True, exist_ok=True)

WIND_100M_TIF = OUTPUT_DIR / 'wind_speed_100m_original.tif'
WIND_SCORE_TIF = OUTPUT_DIR / 'wind_suitability_score_original.tif'
UK_BOUNDARY_SHP = BASE_DIR / 'UK_ITL1_Boundaries.shp'
REFERENCE_GRID_PKL = OUTPUT_DIR / 'reference_grid.pkl'
WIND_SPEED_MAP = MAPS_DIR / 'map1_wind_speed_100m_enhanced.png'
WIND_SCORE_MAP = MAPS_DIR / 'map2_wind_suitability_score_enhanced.png'

WIND_THRESHOLD_MIN = 6.0
WIND_THRESHOLD_MAX = 9.0
MAX_SCORE = 10.0
SCORE_SLOPE = MAX_SCORE / (WIND_THRESHOLD_MAX - WIND_THRESHOLD_MIN)
DATA_HEIGHT = 10
TURBINE_HEIGHT = 100
HELLMAN_EXPONENT = 0.143
WIND_EXTRAPOLATION_FACTOR = (TURBINE_HEIGHT / DATA_HEIGHT) ** HELLMAN_EXPONENT
NODATA_VALUE = -9999
CRS_EPSG = 27700

FIG_WIDTH = 14
FIG_HEIGHT = 18
DPI = 300
OCEAN_COLOR = '#e6f2ff'

WIND_COLORS = ['#313695', '#4575b4', '#74add1', '#abd9e9', '#e0f3f8',
               '#ffffbf', '#fee090', '#fdae61', '#f46d43', '#d73027', '#a50026']
SCORE_COLORS = ['#d73027', '#f46d43', '#fdae61', '#fee090', '#ffffbf',
                '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850', '#006837']

WIND_CLASSES = {
    'Excellent': (9.0, float('inf'), '#a50026'),
    'Very Good': (8.0, 9.0, '#f46d43'),
    'Good': (7.0, 8.0, '#fdae61'),
    'Fair': (6.0, 7.0, '#fee090'),
    'Marginal': (5.0, 6.0, '#ffffbf'),
    'Poor': (0.0, 5.0, '#abd9e9'),
}

SCORE_CLASSES = {
    'Excellent': (9.0, 10.01, '>=8.7 m/s', '#006837'),
    'Good': (7.0, 9.0, '8.1-8.7 m/s', '#66bd63'),
    'Fair': (5.0, 7.0, '7.5-8.1 m/s', '#a6d96a'),
    'Marginal': (3.0, 5.0, '6.9-7.5 m/s', '#ffffbf'),
    'Poor': (0.01, 3.0, '6.3-6.9 m/s', '#fdae61'),
    'Unsuitable': (0.0, 0.01, '<6 m/s', '#d73027'),
}

# Load data
print("\nLoading data")
with rasterio.open(WIND_100M_TIF) as src:
    wind_data = src.read(1)
    transform = src.transform
    wind_data[wind_data == NODATA_VALUE] = np.nan

with rasterio.open(WIND_SCORE_TIF) as src:
    score_data = src.read(1)
    score_data[score_data == NODATA_VALUE] = np.nan

uk_boundary = gpd.read_file(UK_BOUNDARY_SHP)
if uk_boundary.crs.to_epsg() != CRS_EPSG:
    uk_boundary = uk_boundary.to_crs(f'EPSG:{CRS_EPSG}')

with open(REFERENCE_GRID_PKL, 'rb') as f:
    ref = pickle.load(f)
extent = ref['extent']
rows = ref['rows']
cols = ref['cols']

print(f"Data shape: {wind_data.shape}")
print(f"Valid wind cells: {(~np.isnan(wind_data)).sum():,}")
print(f"Valid score cells: {(~np.isnan(score_data)).sum():,}")

# Map 1: Wind Speed
print("\n" + "="*80)
print("GENERATING MAP 1: WIND SPEED")
print("="*80)

fig, ax = plt.subplots(figsize=(FIG_WIDTH, FIG_HEIGHT), dpi=DPI)
ax.set_facecolor(OCEAN_COLOR)
ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5, color='gray', zorder=1)

wind_cmap = LinearSegmentedColormap.from_list('wind_enhanced', WIND_COLORS, N=256)
wind_levels = np.arange(3, 11, 0.5)
wind_norm = BoundaryNorm(wind_levels, wind_cmap.N, clip=True)

im = ax.imshow(wind_data, cmap=wind_cmap, norm=wind_norm,
               extent=extent, origin='upper',
               interpolation='bilinear', aspect='equal', zorder=2)

uk_boundary.boundary.plot(ax=ax, color='black', linewidth=1.5, zorder=10)

ax.set_xlabel('Easting (m)', fontsize=14, fontweight='bold')
ax.set_ylabel('Northing (m)', fontsize=14, fontweight='bold')
ax.set_title('UK Wind Speed at 100m Hub Height\n(Extrapolated from 10m Seasonal Data)',
             fontsize=18, fontweight='bold', pad=20)

ax.ticklabel_format(style='plain')
ax.tick_params(labelsize=11)

cbar = plt.colorbar(im, ax=ax, pad=0.02, fraction=0.046, ticks=[3, 4, 5, 6, 7, 8, 9, 10])
cbar.set_label('Wind Speed (m/s)', fontsize=13, fontweight='bold')
cbar.ax.tick_params(labelsize=11)

cbar.ax.axhline(y=WIND_THRESHOLD_MIN, color='red', linestyle='--', linewidth=2, alpha=0.7)
cbar.ax.text(1.5, WIND_THRESHOLD_MIN, f'{WIND_THRESHOLD_MIN} m/s\n(Threshold)', 
             fontsize=9, va='center', color='red', fontweight='bold',
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.8, edgecolor='red'))

valid_wind = wind_data[~np.isnan(wind_data)]
legend_elements = []
for class_name, (v_min, v_max, color) in WIND_CLASSES.items():
    if v_max == float('inf'):
        mask = valid_wind >= v_min
        label = f'{class_name} (>={v_min} m/s)'
    else:
        mask = (valid_wind >= v_min) & (valid_wind < v_max)
        label = f'{class_name} ({v_min}-{v_max} m/s)'
    pct = mask.sum() / len(valid_wind) * 100
    legend_elements.append(mpatches.Patch(color=color, label=f'{label}: {pct:.1f}%'))

legend = ax.legend(handles=legend_elements, loc='upper left',
                   fontsize=10, framealpha=0.95, edgecolor='black',
                   title='Wind Speed Classification', title_fontsize=11,
                   fancybox=True, shadow=True)
legend.get_frame().set_linewidth(1.5)

stats_text = f"""Statistics (100m):
Mean: {valid_wind.mean():.2f} m/s
Min: {valid_wind.min():.2f} m/s
Max: {valid_wind.max():.2f} m/s
Median: {np.median(valid_wind):.2f} m/s
Std: {valid_wind.std():.2f} m/s

Viable (>={WIND_THRESHOLD_MIN} m/s):
{(valid_wind >= WIND_THRESHOLD_MIN).sum() / len(valid_wind) * 100:.1f}%

Extrapolation:
{DATA_HEIGHT}m -> {TURBINE_HEIGHT}m (alpha={HELLMAN_EXPONENT})
Factor: {WIND_EXTRAPOLATION_FACTOR:.3f}x

Grid: {rows}x{cols}
Coverage: {len(valid_wind):,} cells
({len(valid_wind)/(rows*cols)*100:.1f}%)"""

props = dict(boxstyle='round', facecolor='white', alpha=0.95, edgecolor='black', linewidth=1.5)
ax.text(0.02, 0.15, stats_text, transform=ax.transAxes,
        fontsize=9, verticalalignment='bottom', horizontalalignment='left',
        bbox=props, fontfamily='monospace')

scale_x = extent[0] + (extent[1] - extent[0]) * 0.05
scale_y = extent[2] + (extent[3] - extent[2]) * 0.05
scale_length = 100000

ax.plot([scale_x, scale_x + scale_length], [scale_y, scale_y], 'k-', linewidth=3, zorder=15)
ax.plot([scale_x, scale_x], [scale_y - 5000, scale_y + 5000], 'k-', linewidth=3, zorder=15)
ax.plot([scale_x + scale_length, scale_x + scale_length],
        [scale_y - 5000, scale_y + 5000], 'k-', linewidth=3, zorder=15)
ax.text(scale_x + scale_length/2, scale_y + 12000, '100 km',
        ha='center', fontsize=11, fontweight='bold',
        bbox=dict(boxstyle='round', facecolor='white', alpha=0.9), zorder=15)

arrow_x = extent[1] - (extent[1] - extent[0]) * 0.05
arrow_y = extent[3] - (extent[3] - extent[2]) * 0.08
ax.annotate('N', xy=(arrow_x, arrow_y + 40000), xytext=(arrow_x, arrow_y),
            arrowprops=dict(arrowstyle='->', lw=3, color='black'),
            fontsize=16, fontweight='bold', ha='center', zorder=15)

plt.tight_layout()
plt.savefig(WIND_SPEED_MAP, dpi=DPI, bbox_inches='tight', facecolor='white')
print(f"Saved: {WIND_SPEED_MAP}")
plt.close()

# Map 2: Suitability Score
print("\n" + "="*80)
print("GENERATING MAP 2: SUITABILITY SCORE")
print("="*80)

fig, ax = plt.subplots(figsize=(FIG_WIDTH, FIG_HEIGHT), dpi=DPI)
ax.set_facecolor(OCEAN_COLOR)
ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5, color='gray', zorder=1)

score_cmap = LinearSegmentedColormap.from_list('score_enhanced', SCORE_COLORS, N=256)
score_levels = np.arange(0, 10.5, 0.5)
score_norm = BoundaryNorm(score_levels, score_cmap.N, clip=True)

im = ax.imshow(score_data, cmap=score_cmap, norm=score_norm,
               extent=extent, origin='upper',
               interpolation='bilinear', aspect='equal', zorder=2)

uk_boundary.boundary.plot(ax=ax, color='black', linewidth=1.5, zorder=10)

ax.set_xlabel('Easting (m)', fontsize=14, fontweight='bold')
ax.set_ylabel('Northing (m)', fontsize=14, fontweight='bold')
ax.set_title('UK Wind Speed Suitability Score (0-10)\n(Based on 100m Hub Height Wind Speed)',
             fontsize=18, fontweight='bold', pad=20)

ax.ticklabel_format(style='plain')
ax.tick_params(labelsize=11)

cbar = plt.colorbar(im, ax=ax, pad=0.02, fraction=0.046, ticks=[0, 2, 4, 6, 8, 10])
cbar.set_label('Suitability Score', fontsize=13, fontweight='bold')
cbar.ax.tick_params(labelsize=11)

threshold_info = [
    (0, f'0 (v<{WIND_THRESHOLD_MIN})', '#d73027'),
    (3.33, '3.3 (v=7)', '#fdae61'),
    (6.67, '6.7 (v=8)', '#a6d96a'),
    (10, f'10 (v>={WIND_THRESHOLD_MAX})', '#006837'),
]

for score, label, color in threshold_info:
    cbar.ax.axhline(y=score, color=color, linestyle='--', linewidth=1.5, alpha=0.7)
    cbar.ax.text(1.3, score, label, fontsize=8, va='center',
                 color=color, fontweight='bold',
                 bbox=dict(boxstyle='round', facecolor='white', alpha=0.8, 
                          edgecolor=color, linewidth=1))

valid_scores = score_data[~np.isnan(score_data)]
legend_elements = []
for class_name, (s_min, s_max, wind_range, color) in SCORE_CLASSES.items():
    if s_max == float('inf'):
        mask = valid_scores >= s_min
        score_range = f'>={s_min:.0f}'
    else:
        mask = (valid_scores >= s_min) & (valid_scores < s_max)
        score_range = f'{s_min:.0f}-{s_max:.0f}'
    pct = mask.sum() / len(valid_scores) * 100
    legend_elements.append(
        mpatches.Patch(color=color, 
                      label=f'{class_name} ({score_range}): v {wind_range} - {pct:.1f}%'))

legend = ax.legend(handles=legend_elements, loc='upper left',
                   fontsize=10, framealpha=0.95, edgecolor='black',
                   title='Suitability Classes', title_fontsize=11,
                   fancybox=True, shadow=True)
legend.get_frame().set_linewidth(1.5)

score_dist = [
    (0, 0.01, 'Zero'),
    (0.01, 3, 'Low'),
    (3, 6, 'Medium'),
    (6, 9, 'High'),
    (9, 10.01, 'V.High'),
]

dist_text = "Distribution:\n"
for s_min, s_max, label in score_dist:
    mask = (valid_scores >= s_min) & (valid_scores < s_max)
    pct = mask.sum() / len(valid_scores) * 100
    dist_text += f"{label:>7s}: {pct:5.1f}%\n"

stats_text = f"""Statistics (Score):
Mean: {valid_scores.mean():.2f}
Min: {valid_scores.min():.2f}
Max: {valid_scores.max():.2f}
Median: {np.median(valid_scores):.2f}
Std: {valid_scores.std():.2f}

{dist_text}
Scoring Formula:
v<{WIND_THRESHOLD_MIN}: s=0
{WIND_THRESHOLD_MIN}<=v<={WIND_THRESHOLD_MAX}: s={SCORE_SLOPE:.2f}(v-{WIND_THRESHOLD_MIN})
v>{WIND_THRESHOLD_MAX}: s={MAX_SCORE}

WLC Weight: 39.4%

Grid: {rows}x{cols}
Coverage: {len(valid_scores):,} cells"""

props = dict(boxstyle='round', facecolor='white', alpha=0.95, edgecolor='black', linewidth=1.5)
ax.text(0.02, 0.15, stats_text, transform=ax.transAxes,
        fontsize=8.5, verticalalignment='bottom', horizontalalignment='left',
        bbox=props, fontfamily='monospace')

ax.plot([scale_x, scale_x + scale_length], [scale_y, scale_y], 'k-', linewidth=3, zorder=15)
ax.plot([scale_x, scale_x], [scale_y - 5000, scale_y + 5000], 'k-', linewidth=3, zorder=15)
ax.plot([scale_x + scale_length, scale_x + scale_length],
        [scale_y - 5000, scale_y + 5000], 'k-', linewidth=3, zorder=15)
ax.text(scale_x + scale_length/2, scale_y + 12000, '100 km',
        ha='center', fontsize=11, fontweight='bold',
        bbox=dict(boxstyle='round', facecolor='white', alpha=0.9), zorder=15)

ax.annotate('N', xy=(arrow_x, arrow_y + 40000), xytext=(arrow_x, arrow_y),
            arrowprops=dict(arrowstyle='->', lw=3, color='black'),
            fontsize=16, fontweight='bold', ha='center', zorder=15)

plt.tight_layout()
plt.savefig(WIND_SCORE_MAP, dpi=DPI, bbox_inches='tight', facecolor='white')
print(f"Saved: {WIND_SCORE_MAP}")
plt.close()

print("\n" + "="*80)
print("VISUALIZATION COMPLETE")
print("="*80)
print(f"Output files:")
print(f"  {WIND_SPEED_MAP}")
print(f"  {WIND_SCORE_MAP}")

WIND SPEED VISUALIZATION

Loading data
Data shape: (235, 131)
Valid wind cells: 10,397
Valid score cells: 10,397

GENERATING MAP 1: WIND SPEED
Saved: D:\BENV0093\outputs\maps\map1_wind_speed_100m_enhanced.png

GENERATING MAP 2: SUITABILITY SCORE
Saved: D:\BENV0093\outputs\maps\map2_wind_suitability_score_enhanced.png

VISUALIZATION COMPLETE
Output files:
  D:\BENV0093\outputs\maps\map1_wind_speed_100m_enhanced.png
  D:\BENV0093\outputs\maps\map2_wind_suitability_score_enhanced.png


Grid Distance Processing
Calculates distance to high-voltage transmission lines

In [4]:
import numpy as np
import geopandas as gpd
import rasterio
from rasterio.transform import Affine
from scipy.spatial import cKDTree
import pickle
from pathlib import Path
from shapely.geometry import Point
import time

print("="*80)
print("GRID DISTANCE PROCESSING (PRECISE VECTOR METHOD)")
print("With EXPONENTIAL DECAY SCORING (λ=7.21)")
print("="*80)

BASE_DIR = Path('D:/BENV0093')
DATASET_DIR = BASE_DIR / 'Dateset_T2'
OUTPUT_DIR = BASE_DIR / 'outputs'

TRANSMISSION_LINES = DATASET_DIR / 'TransmissionLines' / 'uk_transmission_lines.geojson'
UK_BOUNDARY_SHP = BASE_DIR / 'UK_ITL1_Boundaries.shp'
REFERENCE_GRID_PKL = OUTPUT_DIR / 'reference_grid.pkl'

GRID_DISTANCE_TIF = OUTPUT_DIR / 'grid_distance.tif'
GRID_SCORE_TIF = OUTPUT_DIR / 'grid_suitability_score.tif'

# NEW: Exponential decay parameters
LAMBDA_DECAY = 7.21  # Decay parameter in km
MAX_SCORE = 10.0
NODATA_VALUE = -9999
CRS_EPSG = 27700

if not TRANSMISSION_LINES.exists():
    print(f"ERROR: Transmission lines not found: {TRANSMISSION_LINES}")
    exit(1)

# Step 1: Load reference grid
print("\nStep 1: Loading reference grid")
with open(REFERENCE_GRID_PKL, 'rb') as f:
    ref = pickle.load(f)

rows, cols = ref['shape']
pixel_size = ref['pixel_size']
x_spacing, y_spacing = pixel_size
extent = ref['extent']

x_min = extent[0]
y_max = extent[3]

transform = Affine(
    pixel_size[0], 0, extent[0],
    0, -abs(pixel_size[1]), extent[3]
)

print(f"Shape: {rows} x {cols}")
print(f"Pixel size: {x_spacing:.2f} x {y_spacing:.2f} m")
print(f"Extent: X[{extent[0]:.0f}, {extent[1]:.0f}], Y[{extent[2]:.0f}, {extent[3]:.0f}]")

# Step 2: Load transmission lines
print("\nStep 2: Loading transmission lines")
transmission_lines = gpd.read_file(TRANSMISSION_LINES)
if transmission_lines.crs.to_epsg() != CRS_EPSG:
    transmission_lines = transmission_lines.to_crs(f'EPSG:{CRS_EPSG}')
print(f"Loaded {len(transmission_lines)} transmission lines")

# Step 3: Extract line coordinates and build KDTree
print("\nStep 3: Extracting line coordinates and building spatial index")

all_coords = []
for idx, line in transmission_lines.iterrows():
    geom = line.geometry
    if geom.geom_type == 'LineString':
        coords = np.array(geom.coords)
        all_coords.append(coords)
    elif geom.geom_type == 'MultiLineString':
        for g in geom.geoms:
            coords = np.array(g.coords)
            all_coords.append(coords)

line_points = np.vstack(all_coords)
print(f"Extracted {len(line_points):,} coordinate points from lines")

print("Densifying line segments...")
densified_points = []
densify_interval = 100

for coords_array in all_coords:
    for i in range(len(coords_array) - 1):
        p1 = coords_array[i]
        p2 = coords_array[i + 1]
        
        segment_length = np.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2)
        n_points = max(1, int(segment_length / densify_interval))
        
        for j in range(n_points + 1):
            t = j / n_points
            point = p1 + t * (p2 - p1)
            densified_points.append(point)

line_points_dense = np.array(densified_points)
print(f"Densified to {len(line_points_dense):,} points ({densify_interval}m interval)")

print("Building KDTree spatial index...")
tree = cKDTree(line_points_dense)
print("KDTree built successfully")

# Step 4: Calculate precise distance for each grid cell center
print("\nStep 4: Calculating precise Euclidean distance for each cell center")
print("This may take a few minutes...")

distance_meters = np.full((rows, cols), np.nan, dtype=np.float32)

start_time = time.time()
processed = 0
total_cells = rows * cols

batch_size = 10000
batch_coords = []
batch_indices = []

for r in range(rows):
    for c in range(cols):
        x_center = x_min + (c + 0.5) * x_spacing
        y_center = y_max - (r + 0.5) * y_spacing
        
        batch_coords.append([x_center, y_center])
        batch_indices.append((r, c))
        
        if len(batch_coords) >= batch_size:
            distances, _ = tree.query(batch_coords)
            
            for (row_idx, col_idx), dist in zip(batch_indices, distances):
                distance_meters[row_idx, col_idx] = dist
            
            batch_coords = []
            batch_indices = []
            
            processed += batch_size
            if processed % 100000 == 0:
                elapsed = time.time() - start_time
                progress = processed / total_cells * 100
                print(f"  Progress: {progress:.1f}% ({processed:,}/{total_cells:,}) - {elapsed:.0f}s elapsed")

if batch_coords:
    distances, _ = tree.query(batch_coords)
    for (row_idx, col_idx), dist in zip(batch_indices, distances):
        distance_meters[row_idx, col_idx] = dist

elapsed = time.time() - start_time
print(f"Distance calculation complete in {elapsed:.0f} seconds")
print(f"Distance range: {np.nanmin(distance_meters):.1f} - {np.nanmax(distance_meters):.1f} m")

# Step 5: Calculate suitability scores using EXPONENTIAL DECAY
print("\nStep 5: Calculating suitability scores")
print(f"Using EXPONENTIAL DECAY scoring:")
print(f"  Formula: Score = {MAX_SCORE} × exp(-distance / λ)")
print(f"  λ (lambda) = {LAMBDA_DECAY} km")
print(f"  ")
print(f"  Distance examples:")
print(f"    0 km   → Score = {MAX_SCORE * np.exp(-0/LAMBDA_DECAY):.2f}")
print(f"    {LAMBDA_DECAY:.2f} km → Score = {MAX_SCORE * np.exp(-1):.2f} (≈37% of max)")
print(f"    {LAMBDA_DECAY*2:.2f} km → Score = {MAX_SCORE * np.exp(-2):.2f} (≈14% of max)")
print(f"    {LAMBDA_DECAY*3:.2f} km → Score = {MAX_SCORE * np.exp(-3):.2f} (≈5% of max)")

suitability_score = np.zeros_like(distance_meters, dtype=np.float32)

# Create valid mask (non-NaN values)
valid_mask = ~np.isnan(distance_meters)

# Convert distance to km
distance_km = distance_meters / 1000.0

# Apply exponential decay: Score = MAX_SCORE × exp(-distance_km / lambda)
suitability_score[valid_mask] = MAX_SCORE * np.exp(-distance_km[valid_mask] / LAMBDA_DECAY)

# Statistics on score distribution
print(f"\nScore distribution:")
for score_threshold in [10, 8, 6, 4, 2, 1, 0.5]:
    count = np.sum(suitability_score[valid_mask] >= score_threshold)
    pct = count / valid_mask.sum() * 100
    if count > 0:
        # Find distance range for this score
        cells_at_threshold = distance_km[valid_mask & (suitability_score >= score_threshold)]
        max_dist = cells_at_threshold.max()
        print(f"  Score ≥ {score_threshold:4.1f}: {count:7,} cells ({pct:5.1f}%) [d ≤ {max_dist:5.1f} km]")

print(f"\nMean score: {suitability_score[valid_mask].mean():.3f}")
print(f"Median score: {np.median(suitability_score[valid_mask]):.3f}")

# Step 6: Apply UK land mask
print("\nStep 6: Applying UK land mask")
uk_boundary = gpd.read_file(UK_BOUNDARY_SHP)
if uk_boundary.crs.to_epsg() != CRS_EPSG:
    uk_boundary = uk_boundary.to_crs(f'EPSG:{CRS_EPSG}')

print("Creating land mask (this may take a moment)...")
land_mask = np.zeros((rows, cols), dtype=bool)

uk_dissolved = uk_boundary.dissolve()
uk_geom = uk_dissolved.geometry.iloc[0]

batch_points = []
batch_indices = []
batch_size_mask = 50000

for r in range(rows):
    for c in range(cols):
        y = y_max - r * y_spacing - y_spacing / 2
        x = x_min + c * x_spacing + x_spacing / 2
        
        batch_points.append(Point(x, y))
        batch_indices.append((r, c))
        
        if len(batch_points) >= batch_size_mask:
            for point, (row_idx, col_idx) in zip(batch_points, batch_indices):
                if uk_geom.contains(point):
                    land_mask[row_idx, col_idx] = True
            
            batch_points = []
            batch_indices = []

if batch_points:
    for point, (row_idx, col_idx) in zip(batch_points, batch_indices):
        if uk_geom.contains(point):
            land_mask[row_idx, col_idx] = True

distance_meters[~land_mask] = np.nan
suitability_score[~land_mask] = np.nan

valid_cells = (~np.isnan(distance_meters)).sum()
print(f"Valid land cells: {valid_cells:,} ({valid_cells/(rows*cols)*100:.1f}%)")

# Step 7: Statistics
print("\nStep 7: Statistics")
valid_dist = distance_meters[~np.isnan(distance_meters)] / 1000
valid_scores = suitability_score[~np.isnan(suitability_score)]

print(f"\nDistance statistics (km):")
print(f"  Min: {valid_dist.min():.3f}")
print(f"  Max: {valid_dist.max():.3f}")
print(f"  Mean: {valid_dist.mean():.3f}")
print(f"  Median: {np.median(valid_dist):.3f}")
print(f"  Std: {valid_dist.std():.3f}")

print(f"\nScore statistics:")
print(f"  Min: {valid_scores.min():.3f}")
print(f"  Max: {valid_scores.max():.3f}")
print(f"  Mean: {valid_scores.mean():.3f}")
print(f"  Median: {np.median(valid_scores):.3f}")
print(f"  Std: {valid_scores.std():.3f}")

unique_distances = np.unique(valid_dist)
print(f"\nNumber of unique distance values: {len(unique_distances):,}")
if len(unique_distances) < 100:
    print("WARNING: Very few unique values detected")
    print(f"Sample values: {unique_distances[:10]}")
else:
    print(f"✓ Continuous distance values detected (good!)")
    print(f"Sample values: {np.sort(valid_dist)[:10]}")

# Show score-distance relationship examples
print(f"\nScore-Distance relationship (exponential decay with λ={LAMBDA_DECAY}):")
example_distances = [0, 1, 2, 5, 7.21, 10, 15, 20, 30]
for d in example_distances:
    score = MAX_SCORE * np.exp(-d / LAMBDA_DECAY)
    print(f"  Distance {d:5.2f} km → Score {score:5.2f}")

# Step 8: Save GeoTIFF files
print("\nStep 8: Saving GeoTIFF files")
metadata = {
    'driver': 'GTiff',
    'dtype': 'float32',
    'nodata': NODATA_VALUE,
    'width': cols,
    'height': rows,
    'count': 1,
    'crs': f'EPSG:{CRS_EPSG}',
    'transform': transform,
    'compress': 'lzw'
}

distance_save = np.where(np.isnan(distance_meters), NODATA_VALUE, distance_meters)
with rasterio.open(GRID_DISTANCE_TIF, 'w', **metadata) as dst:
    dst.write(distance_save.astype('float32'), 1)
print(f"Saved: {GRID_DISTANCE_TIF}")

score_save = np.where(np.isnan(suitability_score), NODATA_VALUE, suitability_score)
with rasterio.open(GRID_SCORE_TIF, 'w', **metadata) as dst:
    dst.write(score_save.astype('float32'), 1)
print(f"Saved: {GRID_SCORE_TIF}")

print("\n" + "="*80)
print("PROCESSING COMPLETE - EXPONENTIAL DECAY SCORING")
print("="*80)
print(f"\nScoring method: Exponential decay")
print(f"  Formula: Score = {MAX_SCORE} × exp(-d / {LAMBDA_DECAY})")
print(f"  λ parameter: {LAMBDA_DECAY} km")
print(f"  ")
print(f"Advantages of exponential decay:")
print(f"  • Smooth continuous scoring (no discrete thresholds)")
print(f"  • Reflects real-world cost curves (exponential growth)")
print(f"  • At λ distance ({LAMBDA_DECAY} km), score drops to 37% of maximum")
print(f"  • Natural decay - score asymptotically approaches zero")
print(f"  ")
print(f"Distance statistics: {valid_dist.min():.1f} - {valid_dist.max():.1f} km (mean: {valid_dist.mean():.1f} km)")
print(f"Score statistics: {valid_scores.min():.2f} - {valid_scores.max():.2f} (mean: {valid_scores.mean():.2f})")
print("="*80)

GRID DISTANCE PROCESSING (PRECISE VECTOR METHOD)
With EXPONENTIAL DECAY SCORING (λ=7.21)

Step 1: Loading reference grid
Shape: 235 x 131
Pixel size: 5000.00 x 5000.00 m
Extent: X[0, 655000], Y[10000, 1185000]

Step 2: Loading transmission lines


C:\Users\888\anaconda3\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: Several features with id = 3993385 have been found. Altering it to be unique. This warning will not be emitted anymore for this layer
  return ogr_read(


Loaded 14654 transmission lines

Step 3: Extracting line coordinates and building spatial index
Extracted 226,154 coordinate points from lines
Densifying line segments...
Densified to 603,492 points (100m interval)
Building KDTree spatial index...
KDTree built successfully

Step 4: Calculating precise Euclidean distance for each cell center
This may take a few minutes...
Distance calculation complete in 0 seconds
Distance range: 3.6 - 285397.2 m

Step 5: Calculating suitability scores
Using EXPONENTIAL DECAY scoring:
  Formula: Score = 10.0 × exp(-distance / λ)
  λ (lambda) = 7.21 km
  
  Distance examples:
    0 km   → Score = 10.00
    7.21 km → Score = 3.68 (≈37% of max)
    14.42 km → Score = 1.35 (≈14% of max)
    21.63 km → Score = 0.50 (≈5% of max)

Score distribution:
  Score ≥  8.0:   2,709 cells (  8.8%) [d ≤   1.6 km]
  Score ≥  6.0:   5,194 cells ( 16.9%) [d ≤   3.7 km]
  Score ≥  4.0:   7,326 cells ( 23.8%) [d ≤   6.6 km]
  Score ≥  2.0:   9,601 cells ( 31.2%) [d ≤  11.6 k

Grid Distance Visualization

In [5]:
import numpy as np
import geopandas as gpd
import rasterio
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.patches as mpatches
import pickle
from pathlib import Path

print("="*80)
print("GRID DISTANCE VISUALIZATION")
print("Exponential Decay Scoring (λ=7.21 km)")
print("="*80)

BASE_DIR = Path('D:/BENV0093')
OUTPUT_DIR = BASE_DIR / 'outputs'

GRID_DISTANCE_TIF = OUTPUT_DIR / 'grid_distance.tif'
GRID_SCORE_TIF = OUTPUT_DIR / 'grid_suitability_score.tif'
UK_BOUNDARY_SHP = BASE_DIR / 'UK_ITL1_Boundaries.shp'
TRANSMISSION_LINES = BASE_DIR / 'Dateset_T2' / 'TransmissionLines' / 'uk_transmission_lines.geojson'
REFERENCE_GRID_PKL = OUTPUT_DIR / 'reference_grid.pkl'

MAP1_FILE = OUTPUT_DIR / 'map3_grid_distance.png'
MAP2_FILE = OUTPUT_DIR / 'map4_grid_suitability_score.png'

NODATA_VALUE = -9999
CRS_EPSG = 27700
LAMBDA_DECAY = 7.21  # Exponential decay parameter

# Load reference grid
print("\nLoading reference grid")
with open(REFERENCE_GRID_PKL, 'rb') as f:
    ref_grid = pickle.load(f)

rows, cols = ref_grid['shape']
pixel_size = ref_grid['pixel_size']
extent = ref_grid['extent']

print(f"Shape: {rows} x {cols}")
print(f"Extent: X[{extent[0]:.0f}, {extent[1]:.0f}], Y[{extent[2]:.0f}, {extent[3]:.0f}]")

# Load rasters
print("\nLoading raster data")
with rasterio.open(GRID_DISTANCE_TIF) as src:
    distance_meters = src.read(1)
    distance_meters[distance_meters == NODATA_VALUE] = np.nan

with rasterio.open(GRID_SCORE_TIF) as src:
    suitability_score = src.read(1)
    suitability_score[suitability_score == NODATA_VALUE] = np.nan

distance_km = distance_meters / 1000

# Load vectors
print("Loading vector data")
uk_boundary = gpd.read_file(UK_BOUNDARY_SHP)
if uk_boundary.crs.to_epsg() != CRS_EPSG:
    uk_boundary = uk_boundary.to_crs(f'EPSG:{CRS_EPSG}')

transmission_lines = gpd.read_file(TRANSMISSION_LINES)
if transmission_lines.crs.to_epsg() != CRS_EPSG:
    transmission_lines = transmission_lines.to_crs(f'EPSG:{CRS_EPSG}')

print(f"Raster: {rows} x {cols}")
print(f"HV Lines: {len(transmission_lines)}")

# Map 1: Distance to Transmission Lines
print("\n" + "="*80)
print("MAP 1: Distance to Transmission Lines")
print("="*80)

fig, ax = plt.subplots(figsize=(14, 18), dpi=300)
ax.set_facecolor('#e6f2ff')
ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5, color='gray', zorder=1)

distance_colors = [
    '#000080', '#0000CD', '#4169E1', '#1E90FF', '#87CEEB',
    '#9370DB', '#BA55D3', '#FF69B4', '#FF6347', '#FF8C00', '#FFA500'
]
dist_cmap = LinearSegmentedColormap.from_list('distance_smooth', distance_colors, N=256)

im = ax.imshow(distance_km, cmap=dist_cmap,
               extent=extent, origin='upper',
               vmin=0, vmax=60,
               interpolation='bilinear', zorder=2)

uk_boundary.boundary.plot(ax=ax, color='black', linewidth=1.5, zorder=10)
transmission_lines.plot(ax=ax, color='darkred', linewidth=0.4, alpha=0.6, zorder=11)

ax.set_xlabel('Easting (m)', fontsize=14, fontweight='bold')
ax.set_ylabel('Northing (m)', fontsize=14, fontweight='bold')
ax.set_title('Distance to Nearest High-Voltage Transmission Line\n(Precise Vector-Based Calculation)',
             fontsize=18, fontweight='bold', pad=20)
ax.ticklabel_format(style='plain')
ax.tick_params(labelsize=11)

cbar = plt.colorbar(im, ax=ax, pad=0.02, fraction=0.046, shrink=0.8)
cbar.set_label('Distance to Grid (km)', fontsize=13, fontweight='bold', labelpad=10)
cbar.ax.tick_params(labelsize=10)

# Add threshold lines at key distances
cbar.ax.axhline(y=LAMBDA_DECAY, color='yellow', linestyle='--', linewidth=2, alpha=0.8)
cbar.ax.text(1.35, LAMBDA_DECAY, f'λ={LAMBDA_DECAY}km', fontsize=8, va='center',
             bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7))

valid_dist = distance_km[~np.isnan(distance_km)]
legend_elements = [
    mpatches.Patch(color='#0000CD', label=f'Very close (0-2 km): {((valid_dist <= 2).sum()/len(valid_dist)*100):.1f}%'),
    mpatches.Patch(color='#1E90FF', label=f'Close (2-7 km): {(((valid_dist > 2) & (valid_dist <= 7)).sum()/len(valid_dist)*100):.1f}%'),
    mpatches.Patch(color='#87CEEB', label=f'Moderate (7-15 km): {(((valid_dist > 7) & (valid_dist <= 15)).sum()/len(valid_dist)*100):.1f}%'),
    mpatches.Patch(color='#BA55D3', label=f'Far (15-30 km): {(((valid_dist > 15) & (valid_dist <= 30)).sum()/len(valid_dist)*100):.1f}%'),
    mpatches.Patch(color='#FF6347', label=f'Very far (30-50 km): {(((valid_dist > 30) & (valid_dist <= 50)).sum()/len(valid_dist)*100):.1f}%'),
    mpatches.Patch(color='#FFA500', label=f'Extremely far (>50 km): {((valid_dist > 50).sum()/len(valid_dist)*100):.1f}%'),
]

legend = ax.legend(handles=legend_elements, loc='upper left',
                   fontsize=9, framealpha=0.95, edgecolor='black',
                   title='Distance Classification', title_fontsize=10,
                   fancybox=True, shadow=True)

stats_text = f"""Distance Statistics (km):
Mean:   {valid_dist.mean():.1f}
Median: {np.median(valid_dist):.1f}
Std:    {valid_dist.std():.1f}
Max:    {valid_dist.max():.1f}

Method: Precise Vector
KDTree + 100m densification
Continuous meter-level accuracy

Coverage (land only):
<= 2 km:  {((valid_dist <= 2).sum()/len(valid_dist)*100):.1f}%
<= 7 km:  {((valid_dist <= 7).sum()/len(valid_dist)*100):.1f}%
<= 15 km: {((valid_dist <= 15).sum()/len(valid_dist)*100):.1f}%

Grid: {rows}x{cols} cells
HV Lines: {len(transmission_lines)}
λ decay: {LAMBDA_DECAY} km"""

props = dict(boxstyle='round', facecolor='white', alpha=0.95,
             edgecolor='black', linewidth=1.5)
ax.text(0.02, 0.15, stats_text, transform=ax.transAxes,
        fontsize=9, verticalalignment='bottom', bbox=props,
        fontfamily='monospace')

scale_x = extent[0] + (extent[1] - extent[0]) * 0.05
scale_y = extent[2] + (extent[3] - extent[2]) * 0.05
scale_length = 100000

ax.plot([scale_x, scale_x + scale_length], [scale_y, scale_y], 'k-', linewidth=3, zorder=15)
ax.plot([scale_x, scale_x], [scale_y - 5000, scale_y + 5000], 'k-', linewidth=3, zorder=15)
ax.plot([scale_x + scale_length, scale_x + scale_length],
        [scale_y - 5000, scale_y + 5000], 'k-', linewidth=3, zorder=15)
ax.text(scale_x + scale_length/2, scale_y + 12000, '100 km',
        ha='center', fontsize=11, fontweight='bold',
        bbox=dict(boxstyle='round', facecolor='white', alpha=0.9), zorder=15)

arrow_x = extent[1] - (extent[1] - extent[0]) * 0.05
arrow_y = extent[3] - (extent[3] - extent[2]) * 0.08
ax.annotate('N', xy=(arrow_x, arrow_y + 40000), xytext=(arrow_x, arrow_y),
            arrowprops=dict(arrowstyle='->', lw=3, color='black'),
            fontsize=16, fontweight='bold', ha='center', zorder=15)

plt.tight_layout()
plt.savefig(MAP1_FILE, dpi=300, bbox_inches='tight', facecolor='white')
print(f"Saved: {MAP1_FILE}")
plt.close()

# Map 2: Suitability Score
print("\n" + "="*80)
print("MAP 2: Grid Connection Suitability Score (Exponential Decay)")
print("="*80)

fig, ax = plt.subplots(figsize=(14, 18), dpi=300)
ax.set_facecolor('#e6f2ff')
ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5, color='gray', zorder=1)

suit_colors = [
    '#a50026', '#d73027', '#f46d43', '#fdae61', '#fee08b',
    '#ffffbf', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850', '#006837'
]
suit_cmap = LinearSegmentedColormap.from_list('suitability_smooth', suit_colors, N=256)

im = ax.imshow(suitability_score, cmap=suit_cmap,
               extent=extent, origin='upper',
               vmin=0, vmax=10,
               interpolation='bilinear', zorder=2)

uk_boundary.boundary.plot(ax=ax, color='black', linewidth=1.5, zorder=10)
transmission_lines.plot(ax=ax, color='darkred', linewidth=0.4, alpha=0.6, zorder=11)

ax.set_xlabel('Easting (m)', fontsize=14, fontweight='bold')
ax.set_ylabel('Northing (m)', fontsize=14, fontweight='bold')
ax.set_title('Grid Connection Suitability Score (0-10)\n(Exponential Decay Scoring: λ=7.21 km)',
             fontsize=18, fontweight='bold', pad=20)
ax.ticklabel_format(style='plain')
ax.tick_params(labelsize=11)

cbar = plt.colorbar(im, ax=ax, pad=0.02, fraction=0.046, shrink=0.8)
cbar.set_label('Suitability Score', fontsize=13, fontweight='bold', labelpad=10)
cbar.ax.tick_params(labelsize=10)

# Add key score thresholds
score_thresholds = [
    (10 * np.exp(-1), 'yellow', f'37% (d=λ)'),
    (10 * np.exp(-2), 'orange', f'14% (d=2λ)'),
]
for score, color, label in score_thresholds:
    cbar.ax.axhline(y=score, color=color, linestyle=':', linewidth=1.5, alpha=0.7)

valid_scores = suitability_score[~np.isnan(suitability_score)]

# Calculate distance ranges for score categories
def get_distance_for_score(score):
    """Calculate distance (km) that gives a specific score"""
    if score >= 10:
        return 0
    return -LAMBDA_DECAY * np.log(score / 10)

legend_elements = [
    mpatches.Patch(color='#006837', 
                   label=f'Excellent (8-10): d ≤ {get_distance_for_score(8):.1f} km - {((valid_scores >= 8).sum()/len(valid_scores)*100):.1f}%'),
    mpatches.Patch(color='#66bd63', 
                   label=f'Good (6-8): d = {get_distance_for_score(8):.1f}-{get_distance_for_score(6):.1f} km - {(((valid_scores >= 6) & (valid_scores < 8)).sum()/len(valid_scores)*100):.1f}%'),
    mpatches.Patch(color='#d9ef8b', 
                   label=f'Fair (4-6): d = {get_distance_for_score(6):.1f}-{get_distance_for_score(4):.1f} km - {(((valid_scores >= 4) & (valid_scores < 6)).sum()/len(valid_scores)*100):.1f}%'),
    mpatches.Patch(color='#fdae61', 
                   label=f'Marginal (2-4): d = {get_distance_for_score(4):.1f}-{get_distance_for_score(2):.1f} km - {(((valid_scores >= 2) & (valid_scores < 4)).sum()/len(valid_scores)*100):.1f}%'),
    mpatches.Patch(color='#d73027', 
                   label=f'Poor (0-2): d > {get_distance_for_score(2):.1f} km - {(((valid_scores >= 0) & (valid_scores < 2)).sum()/len(valid_scores)*100):.1f}%'),
]

legend = ax.legend(handles=legend_elements, loc='upper left',
                   fontsize=9, framealpha=0.95, edgecolor='black',
                   title='Suitability Classes', title_fontsize=10,
                   fancybox=True, shadow=True)

stats_text = f"""Score Statistics:
Mean:   {valid_scores.mean():.2f}
Median: {np.median(valid_scores):.2f}
Std:    {valid_scores.std():.2f}
Min:    {valid_scores.min():.2f}
Max:    {valid_scores.max():.2f}

Scoring Formula:
S = 10 × exp(-d / λ)

where:
  S = Suitability score (0-10)
  d = Distance (km)
  λ = {LAMBDA_DECAY} km (decay parameter)

Key distances:
  d = 0 km    → S = 10.00
  d = {LAMBDA_DECAY:.2f} km → S = 3.68 (37%)
  d = {LAMBDA_DECAY*2:.2f} km → S = 1.35 (14%)
  d = {LAMBDA_DECAY*3:.2f} km → S = 0.50 (5%)

Distribution:
Excellent: {((valid_scores >= 8).sum()/len(valid_scores)*100):5.1f}%
Good:      {(((valid_scores >= 6) & (valid_scores < 8)).sum()/len(valid_scores)*100):5.1f}%
Fair:      {(((valid_scores >= 4) & (valid_scores < 6)).sum()/len(valid_scores)*100):5.1f}%
Marginal:  {(((valid_scores >= 2) & (valid_scores < 4)).sum()/len(valid_scores)*100):5.1f}%
Poor:      {((valid_scores < 2).sum()/len(valid_scores)*100):5.1f}%

WLC Weight: 34.75%
Grid: 275kV+400kV
Method: Vector KDTree"""

props = dict(boxstyle='round', facecolor='white', alpha=0.95,
             edgecolor='black', linewidth=1.5)
ax.text(0.02, 0.15, stats_text, transform=ax.transAxes,
        fontsize=8.5, verticalalignment='bottom', bbox=props,
        fontfamily='monospace')

ax.plot([scale_x, scale_x + scale_length], [scale_y, scale_y], 'k-', linewidth=3, zorder=15)
ax.plot([scale_x, scale_x], [scale_y - 5000, scale_y + 5000], 'k-', linewidth=3, zorder=15)
ax.plot([scale_x + scale_length, scale_x + scale_length],
        [scale_y - 5000, scale_y + 5000], 'k-', linewidth=3, zorder=15)
ax.text(scale_x + scale_length/2, scale_y + 12000, '100 km',
        ha='center', fontsize=11, fontweight='bold',
        bbox=dict(boxstyle='round', facecolor='white', alpha=0.9), zorder=15)

ax.annotate('N', xy=(arrow_x, arrow_y + 40000), xytext=(arrow_x, arrow_y),
            arrowprops=dict(arrowstyle='->', lw=3, color='black'),
            fontsize=16, fontweight='bold', ha='center', zorder=15)

plt.tight_layout()
plt.savefig(MAP2_FILE, dpi=300, bbox_inches='tight', facecolor='white')
print(f"Saved: {MAP2_FILE}")
plt.close()

print("\n" + "="*80)
print("VISUALIZATION COMPLETE")
print("="*80)
print(f"\nScoring Method: EXPONENTIAL DECAY")
print(f"  Formula: Score = 10 × exp(-distance / {LAMBDA_DECAY})")
print(f"  λ parameter: {LAMBDA_DECAY} km")
print(f"  WLC Weight: 34.75%")
print(f"  ")
print(f"Output files:")
print(f"  {MAP1_FILE}")
print(f"  {MAP2_FILE}")
print("="*80)

GRID DISTANCE VISUALIZATION
Exponential Decay Scoring (λ=7.21 km)

Loading reference grid
Shape: 235 x 131
Extent: X[0, 655000], Y[10000, 1185000]

Loading raster data
Loading vector data


C:\Users\888\anaconda3\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: Several features with id = 3993385 have been found. Altering it to be unique. This warning will not be emitted anymore for this layer
  return ogr_read(


Raster: 235 x 131
HV Lines: 14654

MAP 1: Distance to Transmission Lines
Saved: D:\BENV0093\outputs\map3_grid_distance.png

MAP 2: Grid Connection Suitability Score (Exponential Decay)
Saved: D:\BENV0093\outputs\map4_grid_suitability_score.png

VISUALIZATION COMPLETE

Scoring Method: EXPONENTIAL DECAY
  Formula: Score = 10 × exp(-distance / 7.21)
  λ parameter: 7.21 km
  WLC Weight: 34.75%
  
Output files:
  D:\BENV0093\outputs\map3_grid_distance.png
  D:\BENV0093\outputs\map4_grid_suitability_score.png


Roads Distance Processing
Calculates distance to major roads and suitability scores

In [7]:
import numpy as np
import geopandas as gpd
import rasterio
from rasterio.transform import Affine
from scipy.spatial import cKDTree  # NEW: Replace EDT with KDTree
import pickle
from pathlib import Path
from shapely.geometry import Point
import time  # NEW: For progress tracking

print("="*80)
print("ROADS DISTANCE PROCESSING (PRECISE VECTOR METHOD)")
print("="*80)

BASE_DIR = Path('D:/BENV0093')
DATASET_DIR = BASE_DIR / 'Dateset_T2'
OUTPUT_DIR = BASE_DIR / 'outputs'

ROADS_GEOJSON = DATASET_DIR / 'RoadNetwork' / 'uk_major_roads_complete.geojson'
UK_BOUNDARY_SHP = BASE_DIR / 'UK_ITL1_Boundaries.shp'
REFERENCE_GRID_PKL = OUTPUT_DIR / 'reference_grid.pkl'

# NEW: Output to different files to avoid overwriting
ROADS_DISTANCE_TIF = OUTPUT_DIR / 'roads_distance.tif'
ROADS_SCORE_TIF = OUTPUT_DIR / 'roads_suitability_score.tif'

# NEW: Refined thresholds for better discrimination
THRESHOLD_NEAR = 1000      # 1 km (was 2-5 km range)
THRESHOLD_FAR = 10000      # 10 km (was 15 km)
MAX_SCORE = 10.0

MAJOR_ROAD_CLASSES = ['motorway', 'motorway_link', 'trunk', 'trunk_link',
                      'primary', 'primary_link', 'Major Road']

NODATA_VALUE = -9999
CRS_EPSG = 27700

if not ROADS_GEOJSON.exists():
    print(f"ERROR: Roads file not found: {ROADS_GEOJSON}")
    exit(1)

# Step 1: Load reference grid
print("\nStep 1: Loading reference grid")
with open(REFERENCE_GRID_PKL, 'rb') as f:
    ref = pickle.load(f)

rows, cols = ref['shape']
pixel_size = ref['pixel_size']
x_spacing, y_spacing = pixel_size
extent = ref['extent']

x_min = extent[0]
y_max = extent[3]

transform = Affine(
    pixel_size[0], 0, extent[0],
    0, -abs(pixel_size[1]), extent[3]
)

print(f"Shape: {rows} x {cols}")
print(f"Pixel size: {x_spacing:.2f} x {y_spacing:.2f} m")
print(f"Extent: X[{extent[0]:.0f}, {extent[1]:.0f}], Y[{extent[2]:.0f}, {extent[3]:.0f}]")

# Step 2: Load and filter major roads
print("\nStep 2: Loading and filtering major roads")
roads = gpd.read_file(ROADS_GEOJSON)
if roads.crs.to_epsg() != CRS_EPSG:
    roads = roads.to_crs(f'EPSG:{CRS_EPSG}')

print(f"Total roads loaded: {len(roads)}")

roads_filtered = roads[roads['road_class'].isin(MAJOR_ROAD_CLASSES)]
print(f"Major roads (motorway, trunk, primary): {len(roads_filtered)}")

roads_clipped = roads_filtered.cx[extent[0]:extent[1], extent[2]:extent[3]]
print(f"Roads within study area: {len(roads_clipped)}")

total_length_km = roads_clipped.geometry.length.sum() / 1000
print(f"Total road length: {total_length_km:.0f} km")

# Step 3: Extract and densify road coordinates (NEW: KDTree method)
print("\nStep 3: Extracting road coordinates for precise distance calculation")

all_coords = []
for idx, road in roads_clipped.iterrows():
    geom = road.geometry
    if geom.geom_type == 'LineString':
        coords = np.array(geom.coords)
        all_coords.append(coords)
    elif geom.geom_type == 'MultiLineString':
        for g in geom.geoms:
            coords = np.array(g.coords)
            all_coords.append(coords)

# Concatenate all coordinates
road_points = np.vstack(all_coords)
print(f"Extracted {len(road_points):,} coordinate points from roads")

# Densify road segments for better accuracy
print("Densifying road segments...")
densified_points = []
densify_interval = 100  # Add point every 100 meters

for coords_array in all_coords:
    for i in range(len(coords_array) - 1):
        p1 = coords_array[i]
        p2 = coords_array[i + 1]
        
        # Calculate segment length
        segment_length = np.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2)
        
        # Number of intermediate points
        n_points = max(1, int(segment_length / densify_interval))
        
        # Generate intermediate points
        for j in range(n_points + 1):
            t = j / n_points
            point = p1 + t * (p2 - p1)
            densified_points.append(point)

road_points_dense = np.array(densified_points)
print(f"Densified to {len(road_points_dense):,} points ({densify_interval}m interval)")

# Build KDTree for fast nearest neighbor search
print("Building KDTree spatial index...")
tree = cKDTree(road_points_dense)
print("KDTree built successfully")

# Step 4: Calculate precise Euclidean distance for each grid cell center (NEW)
print("\nStep 4: Calculating precise Euclidean distance for each cell center")
print("This may take a few minutes...")

distance_meters = np.full((rows, cols), np.nan, dtype=np.float32)

start_time = time.time()
processed = 0
total_cells = rows * cols

# Process in batches for efficiency
batch_size = 10000
batch_coords = []
batch_indices = []

for r in range(rows):
    for c in range(cols):
        # Calculate cell center coordinates
        x_center = x_min + (c + 0.5) * x_spacing
        y_center = y_max - (r + 0.5) * y_spacing
        
        batch_coords.append([x_center, y_center])
        batch_indices.append((r, c))
        
        if len(batch_coords) >= batch_size:
            # Query KDTree for batch
            distances, _ = tree.query(batch_coords)
            
            # Store results
            for (row_idx, col_idx), dist in zip(batch_indices, distances):
                distance_meters[row_idx, col_idx] = dist
            
            batch_coords = []
            batch_indices = []
            
            processed += batch_size
            if processed % 100000 == 0:
                elapsed = time.time() - start_time
                progress = processed / total_cells * 100
                print(f"  Progress: {progress:.1f}% ({processed:,}/{total_cells:,}) - {elapsed:.0f}s elapsed")

# Process remaining cells
if batch_coords:
    distances, _ = tree.query(batch_coords)
    for (row_idx, col_idx), dist in zip(batch_indices, distances):
        distance_meters[row_idx, col_idx] = dist

elapsed = time.time() - start_time
print(f"Distance calculation complete in {elapsed:.0f} seconds")
print(f"Distance range: {np.nanmin(distance_meters):.1f} - {np.nanmax(distance_meters):.1f} m")

# Step 5: Calculate suitability scores with REFINED thresholds
print("\nStep 5: Calculating suitability scores")
print(f"Using refined thresholds:")
print(f"  Excellent (score=10): d ≤ {THRESHOLD_NEAR/1000} km")
print(f"  Linear decay: {THRESHOLD_NEAR/1000} km < d ≤ {THRESHOLD_FAR/1000} km")
print(f"  Unsuitable (score=0): d > {THRESHOLD_FAR/1000} km")

suitability_score = np.zeros_like(distance_meters, dtype=np.float32)

# Create valid mask (non-NaN values)
valid_mask = ~np.isnan(distance_meters)

# Near zone: d <= 1 km → score = 10
mask_near = valid_mask & (distance_meters <= THRESHOLD_NEAR)
suitability_score[mask_near] = MAX_SCORE

# Medium zone: 1 km < d <= 10 km → linear decay
mask_medium = valid_mask & (distance_meters > THRESHOLD_NEAR) & (distance_meters <= THRESHOLD_FAR)
suitability_score[mask_medium] = MAX_SCORE * (THRESHOLD_FAR - distance_meters[mask_medium]) / (THRESHOLD_FAR - THRESHOLD_NEAR)

# Far zone: d > 10 km → score = 0
mask_far = valid_mask & (distance_meters > THRESHOLD_FAR)
suitability_score[mask_far] = 0.0

print(f"Score distribution:")
print(f"  d ≤ {THRESHOLD_NEAR/1000}km (score=10): {mask_near.sum():,} cells")
print(f"  {THRESHOLD_NEAR/1000}km < d ≤ {THRESHOLD_FAR/1000}km (linear decay): {mask_medium.sum():,} cells")
print(f"  d > {THRESHOLD_FAR/1000}km (score=0): {mask_far.sum():,} cells")

# Step 6: Apply UK land mask (keep original logic)
print("\nStep 6: Applying UK land mask")
uk_boundary = gpd.read_file(UK_BOUNDARY_SHP)
if uk_boundary.crs.to_epsg() != CRS_EPSG:
    uk_boundary = uk_boundary.to_crs(f'EPSG:{CRS_EPSG}')

land_mask = np.zeros((rows, cols), dtype=bool)
for idx, region in uk_boundary.iterrows():
    geom = region.geometry
    bounds = geom.bounds
    
    col_start = max(0, int((bounds[0] - x_min) / x_spacing))
    col_end = min(cols, int((bounds[2] - x_min) / x_spacing) + 1)
    row_start = max(0, int((y_max - bounds[3]) / y_spacing))
    row_end = min(rows, int((y_max - bounds[1]) / y_spacing) + 1)
    
    for r in range(row_start, row_end):
        for c in range(col_start, col_end):
            y = y_max - r * y_spacing - y_spacing / 2
            x = x_min + c * x_spacing + x_spacing / 2
            if geom.contains(Point(x, y)):
                land_mask[r, c] = True

distance_meters[~land_mask] = np.nan
suitability_score[~land_mask] = np.nan

valid_cells = (~np.isnan(distance_meters)).sum()
print(f"Valid land cells: {valid_cells:,} ({valid_cells/(rows*cols)*100:.1f}%)")

# Step 7: Statistics (NEW)
print("\nStep 7: Statistics")
valid_dist = distance_meters[~np.isnan(distance_meters)] / 1000
valid_scores = suitability_score[~np.isnan(suitability_score)]

print(f"\nDistance statistics (km):")
print(f"  Min: {valid_dist.min():.3f}")
print(f"  Max: {valid_dist.max():.3f}")
print(f"  Mean: {valid_dist.mean():.3f}")
print(f"  Median: {np.median(valid_dist):.3f}")
print(f"  Std: {valid_dist.std():.3f}")

print(f"\nScore statistics:")
print(f"  Min: {valid_scores.min():.3f}")
print(f"  Max: {valid_scores.max():.3f}")
print(f"  Mean: {valid_scores.mean():.3f}")
print(f"  Median: {np.median(valid_scores):.3f}")
print(f"  Std: {valid_scores.std():.3f}")

# Distribution by distance bins
print(f"\nDistance distribution:")
bins = [0, 1, 2, 5, 10, 15, 20]
for i in range(len(bins)-1):
    count = np.sum((valid_dist >= bins[i]) & (valid_dist < bins[i+1]))
    pct = count / len(valid_dist) * 100
    print(f"  {bins[i]:3.0f}-{bins[i+1]:3.0f} km: {count:7,} ({pct:5.1f}%)")
count_over = np.sum(valid_dist >= bins[-1])
pct_over = count_over / len(valid_dist) * 100
print(f"  ≥{bins[-1]:3.0f} km: {count_over:7,} ({pct_over:5.1f}%)")

# Check for continuous values
unique_distances = np.unique(valid_dist)
print(f"\nNumber of unique distance values: {len(unique_distances):,}")
if len(unique_distances) < 100:
    print("WARNING: Very few unique values detected")
    print(f"Sample values: {unique_distances[:20]}")
else:
    print(f"✓ Continuous distance values detected (good!)")
    print(f"Sample values (first 10): {np.sort(valid_dist)[:10]}")

# Step 8: Save GeoTIFF files
print("\nStep 8: Saving GeoTIFF files")
metadata = {
    'driver': 'GTiff',
    'dtype': 'float32',
    'nodata': NODATA_VALUE,
    'width': cols,
    'height': rows,
    'count': 1,
    'crs': f'EPSG:{CRS_EPSG}',
    'transform': transform,
    'compress': 'lzw'
}

distance_save = np.where(np.isnan(distance_meters), NODATA_VALUE, distance_meters)
with rasterio.open(ROADS_DISTANCE_TIF, 'w', **metadata) as dst:
    dst.write(distance_save.astype('float32'), 1)
print(f"Saved: {ROADS_DISTANCE_TIF}")

score_save = np.where(np.isnan(suitability_score), NODATA_VALUE, suitability_score)
with rasterio.open(ROADS_SCORE_TIF, 'w', **metadata) as dst:
    dst.write(score_save.astype('float32'), 1)
print(f"Saved: {ROADS_SCORE_TIF}")

print("\n" + "="*80)
print("ROADS PROCESSING COMPLETE - PRECISE VECTOR METHOD")
print("="*80)
print(f"Distance: {valid_dist.min():.1f} - {valid_dist.max():.1f} km (mean: {valid_dist.mean():.1f} km)")
print(f"Score: {valid_scores.min():.2f} - {valid_scores.max():.2f} (mean: {valid_scores.mean():.2f})")

print("\n" + "="*80)
print("KEY IMPROVEMENTS")
print("="*80)
print("1. Vector-based distance calculation (KDTree)")
print("   → Precise distances instead of raster quantization")
print(f"   → {len(unique_distances):,} unique values vs ~10 in old method")
print("")
print("2. Refined scoring thresholds")
print(f"   → Optimal: ≤{THRESHOLD_NEAR/1000}km (was 2-5km)")
print(f"   → Acceptable: {THRESHOLD_NEAR/1000}-{THRESHOLD_FAR/1000}km (was up to 15km)")
print("   → Better discrimination for site selection")
print("")
print("3. Maintained grid alignment")
print(f"   → Same {rows}×{cols} grid at {x_spacing}m resolution")
print("   → Compatible with all other criteria layers")
print("="*80)

ROADS DISTANCE PROCESSING (PRECISE VECTOR METHOD)

Step 1: Loading reference grid
Shape: 235 x 131
Pixel size: 5000.00 x 5000.00 m
Extent: X[0, 655000], Y[10000, 1185000]

Step 2: Loading and filtering major roads
Total roads loaded: 120146
Major roads (motorway, trunk, primary): 120146
Roads within study area: 120120
Total road length: 28636 km

Step 3: Extracting road coordinates for precise distance calculation
Extracted 736,721 coordinate points from roads
Densifying road segments...
Densified to 1,268,659 points (100m interval)
Building KDTree spatial index...
KDTree built successfully

Step 4: Calculating precise Euclidean distance for each cell center
This may take a few minutes...
Distance calculation complete in 0 seconds
Distance range: 4.0 - 392162.9 m

Step 5: Calculating suitability scores
Using refined thresholds:
  Excellent (score=10): d ≤ 1.0 km
  Linear decay: 1.0 km < d ≤ 10.0 km
  Unsuitable (score=0): d > 10.0 km
Score distribution:
  d ≤ 1.0km (score=10): 1,836 ce

Roads Distance Visualization

In [8]:
import numpy as np
import geopandas as gpd
import rasterio
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, BoundaryNorm
import matplotlib.patches as mpatches
import pickle
from pathlib import Path

print("="*80)
print("ROADS DISTANCE VISUALIZATION")
print("="*80)

BASE_DIR = Path('D:/BENV0093')
OUTPUT_DIR = BASE_DIR / 'outputs'

ROADS_DISTANCE_TIF = OUTPUT_DIR / 'roads_distance.tif'
ROADS_SCORE_TIF = OUTPUT_DIR / 'roads_suitability_score.tif'
UK_BOUNDARY_SHP = BASE_DIR / 'UK_ITL1_Boundaries.shp'
ROADS_GEOJSON = BASE_DIR / 'Dateset_T2' / 'RoadNetwork' / 'uk_major_roads_complete.geojson'
REFERENCE_GRID_PKL = OUTPUT_DIR / 'reference_grid.pkl'

MAP1_FILE = OUTPUT_DIR / 'map5_roads_distance.png'
MAP2_FILE = OUTPUT_DIR / 'map6_roads_suitability_score.png'

MAJOR_ROAD_CLASSES = ['motorway', 'motorway_link', 'trunk', 'trunk_link',
                      'primary', 'primary_link', 'Major Road']

NODATA_VALUE = -9999
CRS_EPSG = 27700

# Load reference grid
print("\nLoading reference grid")
with open(REFERENCE_GRID_PKL, 'rb') as f:
    ref_grid = pickle.load(f)

rows, cols = ref_grid['shape']
pixel_size = ref_grid['pixel_size']
extent = ref_grid['extent']

print(f"Shape: {rows} x {cols}")
print(f"Extent: X[{extent[0]:.0f}, {extent[1]:.0f}], Y[{extent[2]:.0f}, {extent[3]:.0f}]")

# Load rasters
print("\nLoading raster data")
with rasterio.open(ROADS_DISTANCE_TIF) as src:
    distance_meters = src.read(1)
    distance_meters[distance_meters == NODATA_VALUE] = np.nan

with rasterio.open(ROADS_SCORE_TIF) as src:
    score_data = src.read(1)
    score_data[score_data == NODATA_VALUE] = np.nan

distance_km = distance_meters / 1000

# Load vectors
print("Loading vector data")
uk_boundary = gpd.read_file(UK_BOUNDARY_SHP)
if uk_boundary.crs.to_epsg() != CRS_EPSG:
    uk_boundary = uk_boundary.to_crs(f'EPSG:{CRS_EPSG}')

roads = gpd.read_file(ROADS_GEOJSON)
if roads.crs.to_epsg() != CRS_EPSG:
    roads = roads.to_crs(f'EPSG:{CRS_EPSG}')

roads_filtered = roads[roads['road_class'].isin(MAJOR_ROAD_CLASSES)]
roads_clipped = roads_filtered.cx[extent[0]:extent[1], extent[2]:extent[3]]
roads_sample = roads_clipped.sample(min(3000, len(roads_clipped))) if len(roads_clipped) > 0 else roads_clipped

print(f"Raster: {rows} x {cols}")
print(f"Roads: {len(roads_clipped):,} segments")

# Map 1: Distance to Major Roads
print("\n" + "="*80)
print("MAP 1: Distance to Major Roads")
print("="*80)

fig, ax = plt.subplots(figsize=(14, 18), dpi=300)
ax.set_facecolor('#e6f2ff')
ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5, color='gray', zorder=1)

dist_colors = [
    '#313695', '#4575b4', '#74add1', '#abd9e9', '#e0f3f8',
    '#ffffbf', '#fee090', '#fdae61', '#f46d43', '#d73027', '#a50026'
]
dist_cmap = LinearSegmentedColormap.from_list('distance', dist_colors, N=256)
dist_levels = np.arange(0, 30.5, 0.5)
dist_norm = BoundaryNorm(dist_levels, dist_cmap.N, clip=True)

im = ax.imshow(distance_km, cmap=dist_cmap, norm=dist_norm,
               extent=extent, origin='upper',
               interpolation='bilinear', aspect='equal', zorder=2)

uk_boundary.boundary.plot(ax=ax, color='black', linewidth=1.5, zorder=10)

if len(roads_sample) > 0:
    roads_sample.plot(ax=ax, color='red', linewidth=0.3, alpha=0.5, zorder=11)

ax.set_xlabel('Easting (m)', fontsize=14, fontweight='bold')
ax.set_ylabel('Northing (m)', fontsize=14, fontweight='bold')
ax.set_title('Distance to Nearest Major Road\n(Transport Accessibility for Wind Farm Development)',
             fontsize=18, fontweight='bold', pad=20)
ax.ticklabel_format(style='plain')
ax.tick_params(labelsize=11)

cbar = plt.colorbar(im, ax=ax, pad=0.02, fraction=0.046,
                    ticks=[0, 5, 10, 15, 20, 25, 30])
cbar.set_label('Distance (km)', fontsize=13, fontweight='bold')
cbar.ax.tick_params(labelsize=11)

valid_dist = distance_km[~np.isnan(distance_km)]
d_pct = [
    ((valid_dist<2).sum()/len(valid_dist)*100),
    (((valid_dist>=2)&(valid_dist<5)).sum()/len(valid_dist)*100),
    (((valid_dist>=5)&(valid_dist<10)).sum()/len(valid_dist)*100),
    (((valid_dist>=10)&(valid_dist<15)).sum()/len(valid_dist)*100),
    ((valid_dist>=15).sum()/len(valid_dist)*100)
]

legend_elements = [
    mpatches.Patch(color='#313695', label=f'Very close (<2 km): {d_pct[0]:.1f}%'),
    mpatches.Patch(color='#74add1', label=f'Optimal (2-5 km): {d_pct[1]:.1f}%'),
    mpatches.Patch(color='#ffffbf', label=f'Good (5-10 km): {d_pct[2]:.1f}%'),
    mpatches.Patch(color='#fdae61', label=f'Moderate (10-15 km): {d_pct[3]:.1f}%'),
    mpatches.Patch(color='#d73027', label=f'Far (>15 km): {d_pct[4]:.1f}%'),
]

legend = ax.legend(handles=legend_elements, loc='upper left',
                   fontsize=10, framealpha=0.95, edgecolor='black',
                   title='Distance Classification', title_fontsize=11,
                   fancybox=True, shadow=True)
legend.get_frame().set_linewidth(1.5)

stats_text = f"""Statistics (Distance):
Mean: {valid_dist.mean():.1f} km
Min: {valid_dist.min():.1f} km
Max: {valid_dist.max():.1f} km
Median: {np.median(valid_dist):.1f} km
Std: {valid_dist.std():.1f} km

Accessible (<=15 km):
{(valid_dist <= 15).sum() / len(valid_dist) * 100:.1f}%

Roads Included:
Motorways, trunk,
primary, Major Road

Total: {len(roads_clipped):,} segments
Length: {roads_clipped.geometry.length.sum()/1000:.0f} km

Grid: {rows}x{cols}
Coverage: {len(valid_dist):,} cells
({len(valid_dist)/(rows*cols)*100:.1f}%)"""

props = dict(boxstyle='round', facecolor='white', alpha=0.95,
             edgecolor='black', linewidth=1.5)
ax.text(0.02, 0.15, stats_text, transform=ax.transAxes,
        fontsize=9, verticalalignment='bottom', horizontalalignment='left',
        bbox=props, fontfamily='monospace')

scale_x = extent[0] + (extent[1] - extent[0]) * 0.05
scale_y = extent[2] + (extent[3] - extent[2]) * 0.05
scale_length = 100000

ax.plot([scale_x, scale_x + scale_length], [scale_y, scale_y], 'k-', linewidth=3, zorder=15)
ax.plot([scale_x, scale_x], [scale_y - 5000, scale_y + 5000], 'k-', linewidth=3, zorder=15)
ax.plot([scale_x + scale_length, scale_x + scale_length],
        [scale_y - 5000, scale_y + 5000], 'k-', linewidth=3, zorder=15)
ax.text(scale_x + scale_length/2, scale_y + 12000, '100 km',
        ha='center', fontsize=11, fontweight='bold',
        bbox=dict(boxstyle='round', facecolor='white', alpha=0.9), zorder=15)

arrow_x = extent[1] - (extent[1] - extent[0]) * 0.05
arrow_y = extent[3] - (extent[3] - extent[2]) * 0.08
ax.annotate('N', xy=(arrow_x, arrow_y + 40000), xytext=(arrow_x, arrow_y),
            arrowprops=dict(arrowstyle='->', lw=3, color='black'),
            fontsize=16, fontweight='bold', ha='center', zorder=15)

plt.tight_layout()
plt.savefig(MAP1_FILE, dpi=300, bbox_inches='tight', facecolor='white')
print(f"Saved: {MAP1_FILE}")
plt.close()

# Map 2: Suitability Score
print("\n" + "="*80)
print("MAP 2: Road Proximity Suitability Score")
print("="*80)

fig, ax = plt.subplots(figsize=(14, 18), dpi=300)
ax.set_facecolor('#e6f2ff')
ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5, color='gray', zorder=1)

score_colors = [
    '#d73027', '#f46d43', '#fdae61', '#fee090', '#ffffbf',
    '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850', '#006837'
]
score_cmap = LinearSegmentedColormap.from_list('score', score_colors, N=256)
score_levels = np.arange(0, 10.5, 0.5)
score_norm = BoundaryNorm(score_levels, score_cmap.N, clip=True)

im = ax.imshow(score_data, cmap=score_cmap, norm=score_norm,
               extent=extent, origin='upper',
               interpolation='bilinear', aspect='equal', zorder=2)

uk_boundary.boundary.plot(ax=ax, color='black', linewidth=1.5, zorder=10)

if len(roads_sample) > 0:
    roads_sample.plot(ax=ax, color='darkred', linewidth=0.3, alpha=0.5, zorder=11)

ax.set_xlabel('Easting (m)', fontsize=14, fontweight='bold')
ax.set_ylabel('Northing (m)', fontsize=14, fontweight='bold')
ax.set_title('Road Proximity Suitability Score (0-10)\n(Based on Distance to Major Roads)',
             fontsize=18, fontweight='bold', pad=20)
ax.ticklabel_format(style='plain')
ax.tick_params(labelsize=11)

cbar = plt.colorbar(im, ax=ax, pad=0.02, fraction=0.046,
                    ticks=[0, 2, 4, 6, 8, 10])
cbar.set_label('Suitability Score', fontsize=13, fontweight='bold')
cbar.ax.tick_params(labelsize=11)

threshold_info = [
    (0, '0 (D>15)', '#d73027'),
    (3.33, '3.3 (D=12)', '#fdae61'),
    (6.67, '6.7 (D=8)', '#a6d96a'),
    (10, '10 (2-5km)', '#006837'),
]

for score, label, color in threshold_info:
    cbar.ax.axhline(y=score, color=color, linestyle='--', linewidth=1.5, alpha=0.7)
    cbar.ax.text(1.3, score, label, fontsize=8, va='center',
                 color=color, fontweight='bold',
                 bbox=dict(boxstyle='round', facecolor='white', alpha=0.8,
                          edgecolor=color, linewidth=1))

valid_scores = score_data[~np.isnan(score_data)]
s_pct = [
    ((valid_scores>=9).sum()/len(valid_scores)*100),
    (((valid_scores>=7)&(valid_scores<9)).sum()/len(valid_scores)*100),
    (((valid_scores>=5)&(valid_scores<7)).sum()/len(valid_scores)*100),
    (((valid_scores>=3)&(valid_scores<5)).sum()/len(valid_scores)*100),
    (((valid_scores>0)&(valid_scores<3)).sum()/len(valid_scores)*100),
    ((valid_scores==0).sum()/len(valid_scores)*100)
]

legend_elements = [
    mpatches.Patch(color='#006837', label=f'Excellent (9-10): 2-5 km - {s_pct[0]:.1f}%'),
    mpatches.Patch(color='#66bd63', label=f'Good (7-9): <2 or 6-8 km - {s_pct[1]:.1f}%'),
    mpatches.Patch(color='#a6d96a', label=f'Fair (5-7): 8-10 km - {s_pct[2]:.1f}%'),
    mpatches.Patch(color='#ffffbf', label=f'Marginal (3-5): 10-12 km - {s_pct[3]:.1f}%'),
    mpatches.Patch(color='#fdae61', label=f'Poor (1-3): 12-14 km - {s_pct[4]:.1f}%'),
    mpatches.Patch(color='#d73027', label=f'Unsuitable (0): >15 km - {s_pct[5]:.1f}%'),
]

legend = ax.legend(handles=legend_elements, loc='upper left',
                   fontsize=10, framealpha=0.95, edgecolor='black',
                   title='Suitability Classes', title_fontsize=11,
                   fancybox=True, shadow=True)
legend.get_frame().set_linewidth(1.5)

score_dist = [
    (0, 0.01, 'Zero'),
    (0.01, 3, 'Low'),
    (3, 6, 'Medium'),
    (6, 9, 'High'),
    (9, 10.01, 'V.High'),
]

dist_text = "Distribution:\n"
for s_min, s_max, label in score_dist:
    mask = (valid_scores >= s_min) & (valid_scores < s_max)
    pct = mask.sum() / len(valid_scores) * 100
    dist_text += f"{label:>7s}: {pct:5.1f}%\n"

stats_text = f"""Statistics (Score):
Mean: {valid_scores.mean():.2f}
Min: {valid_scores.min():.2f}
Max: {valid_scores.max():.2f}
Median: {np.median(valid_scores):.2f}
Std: {valid_scores.std():.2f}

{dist_text}
Scoring Formula:
D<2km: s=8
2<=D<=5km: s=10
5<D<=15km: s=15-D
D>15km: s=0

WLC Weight: 8.2%

Grid: {rows}x{cols}
Coverage: {len(valid_scores):,} cells"""

props = dict(boxstyle='round', facecolor='white', alpha=0.95,
             edgecolor='black', linewidth=1.5)
ax.text(0.02, 0.15, stats_text, transform=ax.transAxes,
        fontsize=8.5, verticalalignment='bottom', horizontalalignment='left',
        bbox=props, fontfamily='monospace')

ax.plot([scale_x, scale_x + scale_length], [scale_y, scale_y], 'k-', linewidth=3, zorder=15)
ax.plot([scale_x, scale_x], [scale_y - 5000, scale_y + 5000], 'k-', linewidth=3, zorder=15)
ax.plot([scale_x + scale_length, scale_x + scale_length],
        [scale_y - 5000, scale_y + 5000], 'k-', linewidth=3, zorder=15)
ax.text(scale_x + scale_length/2, scale_y + 12000, '100 km',
        ha='center', fontsize=11, fontweight='bold',
        bbox=dict(boxstyle='round', facecolor='white', alpha=0.9), zorder=15)

ax.annotate('N', xy=(arrow_x, arrow_y + 40000), xytext=(arrow_x, arrow_y),
            arrowprops=dict(arrowstyle='->', lw=3, color='black'),
            fontsize=16, fontweight='bold', ha='center', zorder=15)

plt.tight_layout()
plt.savefig(MAP2_FILE, dpi=300, bbox_inches='tight', facecolor='white')
print(f"Saved: {MAP2_FILE}")
plt.close()

print("\n" + "="*80)
print("VISUALIZATION COMPLETE")
print("="*80)
print(f"Output files:")
print(f"  {MAP1_FILE}")
print(f"  {MAP2_FILE}")

ROADS DISTANCE VISUALIZATION

Loading reference grid
Shape: 235 x 131
Extent: X[0, 655000], Y[10000, 1185000]

Loading raster data
Loading vector data
Raster: 235 x 131
Roads: 120,120 segments

MAP 1: Distance to Major Roads
Saved: D:\BENV0093\outputs\map5_roads_distance.png

MAP 2: Road Proximity Suitability Score
Saved: D:\BENV0093\outputs\map6_roads_suitability_score.png

VISUALIZATION COMPLETE
Output files:
  D:\BENV0093\outputs\map5_roads_distance.png
  D:\BENV0093\outputs\map6_roads_suitability_score.png


Slope Processing
Calculates slope from terrain DEM and suitability scores

In [9]:
import numpy as np
import rasterio
from rasterio.transform import Affine
from rasterio.enums import Resampling
import rasterio.warp
import pickle
from pathlib import Path

print("="*80)
print("SLOPE PROCESSING")
print("="*80)

BASE_DIR = Path('D:/BENV0093')
DATASET_DIR = BASE_DIR / 'Dateset_T2'
OUTPUT_DIR = BASE_DIR / 'outputs'

TERRAIN_TIF = DATASET_DIR / 'Terrain' / 'terrain.tif'
REFERENCE_GRID_PKL = OUTPUT_DIR / 'reference_grid.pkl'
WIND_TIF = OUTPUT_DIR / 'wind_suitability_score_original.tif'

SLOPE_TIF = OUTPUT_DIR / 'slope_degrees.tif'
SLOPE_SCORE_TIF = OUTPUT_DIR / 'slope_suitability_score.tif'

SLOPE_EXCELLENT = 1
SLOPE_MAX = 6
NODATA_VALUE = -9999
CRS_EPSG = 27700

if not TERRAIN_TIF.exists():
    print(f"ERROR: Terrain file not found: {TERRAIN_TIF}")
    exit(1)

# Step 1: Load reference grid
print("\nStep 1: Loading reference grid")
with open(REFERENCE_GRID_PKL, 'rb') as f:
    ref = pickle.load(f)

target_rows, target_cols = ref['shape']
target_pixel_size = ref['pixel_size']
target_extent = ref['extent']

target_transform = Affine(
    target_pixel_size[0], 0, target_extent[0],
    0, -abs(target_pixel_size[1]), target_extent[3]
)

print(f"Target shape: {target_rows} x {target_cols}")
print(f"Target pixel size: {target_pixel_size[0]:.0f} x {abs(target_pixel_size[1]):.0f} m")

# Step 2: Load terrain and calculate slope at native resolution
print("\nStep 2: Loading terrain and calculating slope")
with rasterio.open(TERRAIN_TIF) as src:
    elevation_native = src.read(1).astype('float32')
    native_transform = src.transform
    native_pixel_size = abs(native_transform.a)
    native_crs = src.crs
    
    print(f"Native shape: {elevation_native.shape}")
    print(f"Native pixel size: {native_pixel_size:.0f}m")
    
    elevation_nodata = src.nodata
    if elevation_nodata is not None:
        elevation_native[elevation_native == elevation_nodata] = np.nan
    
    elevation_native[elevation_native < -1000] = np.nan
    elevation_native[elevation_native > 2000] = np.nan

print("Calculating slope at native resolution")
dy_native, dx_native = np.gradient(elevation_native)

dy_per_meter = dy_native / native_pixel_size
dx_per_meter = dx_native / native_pixel_size

slope_radians_native = np.arctan(np.sqrt(dx_per_meter**2 + dy_per_meter**2))
slope_degrees_native = np.degrees(slope_radians_native)

slope_degrees_native[np.isnan(elevation_native)] = np.nan

valid_slope_native = slope_degrees_native[~np.isnan(slope_degrees_native)]
print(f"Native slope: {valid_slope_native.min():.2f} - {valid_slope_native.max():.2f} deg (mean: {valid_slope_native.mean():.2f})")

# Step 3: Resample slope to target grid
print(f"\nStep 3: Resampling from {native_pixel_size:.0f}m to {target_pixel_size[0]:.0f}m")
slope_degrees = np.empty((target_rows, target_cols), dtype=np.float32)

rasterio.warp.reproject(
    source=slope_degrees_native,
    destination=slope_degrees,
    src_transform=native_transform,
    src_crs=native_crs,
    dst_transform=target_transform,
    dst_crs=f'EPSG:{CRS_EPSG}',
    resampling=Resampling.average
)

print(f"Resampled to {target_rows} x {target_cols}")

valid_slope = slope_degrees[~np.isnan(slope_degrees)]
print(f"Slope range: {valid_slope.min():.2f} - {valid_slope.max():.2f} deg (mean: {valid_slope.mean():.2f})")

slope_ranges = [
    (0, 1, "Flat (<=1 deg)"),
    (1, 3, "Gentle (1-3 deg)"),
    (3, 6, "Moderate (3-6 deg)"),
    (6, 10, "Steep (6-10 deg)"),
    (10, np.inf, "Very steep (>10 deg)"),
]

print("\nSlope distribution:")
for s_min, s_max, label in slope_ranges:
    mask = (valid_slope >= s_min) & (valid_slope < s_max)
    pct = mask.sum() / len(valid_slope) * 100
    print(f"  {label:25s}: {pct:5.1f}%")

# Step 4: Apply land mask
print("\nStep 4: Applying land mask")
with rasterio.open(WIND_TIF) as src:
    wind_data = src.read(1)
    land_mask = (wind_data != NODATA_VALUE) & (~np.isnan(wind_data))

slope_degrees[~land_mask] = np.nan

valid_land_cells = land_mask.sum()
print(f"Valid land cells: {valid_land_cells:,} ({valid_land_cells/(target_rows*target_cols)*100:.1f}%)")

# Step 5: Calculate suitability scores
print("\nStep 5: Calculating suitability scores")
print(f"Formula:")
print(f"  S <= 1 deg:    score = 10")
print(f"  1 < S <= 6 deg: score = 10 - 2(S-1)")
print(f"  S > 6 deg:     score = 0")

suitability_score = np.zeros_like(slope_degrees, dtype='float32')

mask1 = slope_degrees <= SLOPE_EXCELLENT
suitability_score[mask1] = 10

mask2 = (slope_degrees > SLOPE_EXCELLENT) & (slope_degrees <= SLOPE_MAX)
suitability_score[mask2] = 10 - (10 / (SLOPE_MAX - SLOPE_EXCELLENT)) * (slope_degrees[mask2] - SLOPE_EXCELLENT)

suitability_score[~land_mask] = np.nan

print(f"S <= {SLOPE_EXCELLENT} deg (score=10): {mask1.sum():,} cells")
print(f"{SLOPE_EXCELLENT} < S <= {SLOPE_MAX} deg (varies): {mask2.sum():,} cells")
print(f"S > {SLOPE_MAX} deg (score=0): {((slope_degrees > SLOPE_MAX) & land_mask).sum():,} cells")

valid_scores = suitability_score[~np.isnan(suitability_score)]
print(f"\nScore statistics:")
print(f"  Mean: {valid_scores.mean():.2f}")
print(f"  Median: {np.median(valid_scores):.2f}")
print(f"  Min: {valid_scores.min():.2f}")
print(f"  Max: {valid_scores.max():.2f}")

score_ranges = [
    (9, 10.01, "Excellent (9-10): <=1 deg"),
    (7, 9, "Good (7-9): 1-2 deg"),
    (5, 7, "Fair (5-7): 2-3.5 deg"),
    (3, 5, "Marginal (3-5): 3.5-5 deg"),
    (1, 3, "Poor (1-3): 5-6 deg"),
    (0, 1, "Unsuitable (0-1): >6 deg"),
]

print("\nScore distribution:")
for s_min, s_max, label in score_ranges:
    mask = (valid_scores >= s_min) & (valid_scores < s_max)
    pct = mask.sum() / len(valid_scores) * 100
    print(f"  {label:30s}: {pct:5.1f}%")

# Step 6: Save GeoTIFF files
print("\nStep 6: Saving GeoTIFF files")
metadata = {
    'driver': 'GTiff',
    'dtype': 'float32',
    'nodata': NODATA_VALUE,
    'width': target_cols,
    'height': target_rows,
    'count': 1,
    'crs': f'EPSG:{CRS_EPSG}',
    'transform': target_transform,
    'compress': 'lzw'
}

slope_save = np.where(np.isnan(slope_degrees), NODATA_VALUE, slope_degrees)
with rasterio.open(SLOPE_TIF, 'w', **metadata) as dst:
    dst.write(slope_save.astype('float32'), 1)
print(f"Saved: {SLOPE_TIF}")

score_save = np.where(np.isnan(suitability_score), NODATA_VALUE, suitability_score)
with rasterio.open(SLOPE_SCORE_TIF, 'w', **metadata) as dst:
    dst.write(score_save.astype('float32'), 1)
print(f"Saved: {SLOPE_SCORE_TIF}")

# Step 7: Verify alignment
print("\nStep 7: Verifying alignment with wind TIF")
with rasterio.open(WIND_TIF) as wind, rasterio.open(SLOPE_SCORE_TIF) as slope:
    checks = {
        'Shape': wind.shape == slope.shape,
        'CRS': wind.crs == slope.crs,
        'Transform': wind.transform.almost_equals(slope.transform, precision=0.01),
    }
    
    for check, result in checks.items():
        status = "MATCH" if result else "MISMATCH"
        print(f"  {check}: {status}")
    
    if all(checks.values()):
        print("\nPERFECT ALIGNMENT")

print("\n" + "="*80)
print("PROCESSING COMPLETE")
print("="*80)
print(f"Slope: {valid_slope.min():.2f} - {valid_slope.max():.2f} deg (mean: {valid_slope.mean():.2f})")
print(f"Score: {valid_scores.min():.2f} - {valid_scores.max():.2f} (mean: {valid_scores.mean():.2f})")

SLOPE PROCESSING

Step 1: Loading reference grid
Target shape: 235 x 131
Target pixel size: 5000 x 5000 m

Step 2: Loading terrain and calculating slope
Native shape: (492, 264)
Native pixel size: 2500m
Calculating slope at native resolution
Native slope: 0.00 - 13.73 deg (mean: 0.39)

Step 3: Resampling from 2500m to 5000m
Resampled to 235 x 131
Slope range: 0.00 - 8.79 deg (mean: 0.41)

Slope distribution:
  Flat (<=1 deg)           :  85.3%
  Gentle (1-3 deg)         :  11.8%
  Moderate (3-6 deg)       :   2.8%
  Steep (6-10 deg)         :   0.1%
  Very steep (>10 deg)     :   0.0%

Step 4: Applying land mask
Valid land cells: 10,397 (33.8%)

Step 5: Calculating suitability scores
Formula:
  S <= 1 deg:    score = 10
  1 < S <= 6 deg: score = 10 - 2(S-1)
  S > 6 deg:     score = 0
S <= 1 deg (score=10): 5,907 cells
1 < S <= 6 deg (varies): 4,463 cells
S > 6 deg (score=0): 27 cells

Score statistics:
  Mean: 8.98
  Median: 10.00
  Min: 0.00
  Max: 10.00

Score distribution:
  Excelle

Slope Visualization
Generates maps for slope and suitability

In [10]:
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.patches as mpatches
import geopandas as gpd
import pickle
from pathlib import Path

print("="*80)
print("SLOPE VISUALIZATION (FIXED)")
print("="*80)

BASE_DIR = Path('D:/BENV0093')
OUTPUT_DIR = BASE_DIR / 'outputs'

SLOPE_TIF = OUTPUT_DIR / 'slope_degrees.tif'
SLOPE_SCORE_TIF = OUTPUT_DIR / 'slope_suitability_score.tif'
UK_BOUNDARY_SHP = BASE_DIR / 'UK_ITL1_Boundaries.shp'
REFERENCE_GRID_PKL = OUTPUT_DIR / 'reference_grid.pkl'

MAP1_FILE = OUTPUT_DIR / 'map7_slope_degrees.png'
MAP2_FILE = OUTPUT_DIR / 'map8_slope_suitability_score.png'

NODATA_VALUE = -9999
CRS_EPSG = 27700

# Load reference grid
print("\nLoading reference grid")
with open(REFERENCE_GRID_PKL, 'rb') as f:
    ref_grid = pickle.load(f)

rows, cols = ref_grid['shape']
extent = ref_grid['extent']

print(f"Shape: {rows} x {cols}")
print(f"Extent: X[{extent[0]:.0f}, {extent[1]:.0f}], Y[{extent[2]:.0f}, {extent[3]:.0f}]")

# Load rasters
print("\nLoading raster data")
with rasterio.open(SLOPE_TIF) as src:
    slope_degrees = src.read(1)
    slope_degrees[slope_degrees == NODATA_VALUE] = np.nan

with rasterio.open(SLOPE_SCORE_TIF) as src:
    suitability_score = src.read(1)
    suitability_score[suitability_score == NODATA_VALUE] = np.nan

# Load UK boundary
print("Loading UK boundary")
uk_boundary = gpd.read_file(UK_BOUNDARY_SHP)
if uk_boundary.crs.to_epsg() != CRS_EPSG:
    uk_boundary = uk_boundary.to_crs(f'EPSG:{CRS_EPSG}')

valid_slope = slope_degrees[~np.isnan(slope_degrees)]
valid_scores = suitability_score[~np.isnan(suitability_score)]

print(f"Valid slope cells: {len(valid_slope):,}")
print(f"Valid score cells: {len(valid_scores):,}")

# Statistics
print(f"\nSlope statistics:")
print(f"  Min: {valid_slope.min():.2f}°")
print(f"  Max: {valid_slope.max():.2f}°")
print(f"  Mean: {valid_slope.mean():.2f}°")
print(f"  Median: {np.median(valid_slope):.2f}°")

# Distribution analysis
print(f"\nSlope distribution:")
for threshold in [1, 2, 3, 6, 10, 15]:
    count = (valid_slope <= threshold).sum()
    pct = count / len(valid_slope) * 100
    print(f"  ≤{threshold:2d}°: {count:7,} cells ({pct:5.1f}%)")

# Map 1: Slope in degrees
print("\n" + "="*80)
print("MAP 1: Slope (Degrees)")
print("="*80)

fig, ax = plt.subplots(figsize=(14, 18), dpi=300)
ax.set_facecolor('#e6f2ff')
ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5, color='gray', zorder=1)

# Colormap for 0-6 degrees (matches vmax)
slope_colors = [
    '#ffffff', '#ffffcc', '#ffeda0', '#fed976', '#feb24c', '#fd8d3c', '#fc4e2a', '#e31a1c', '#bd0026'
]
slope_cmap = LinearSegmentedColormap.from_list('slope', slope_colors, N=256)

# Cap display at reasonable maximum
vmax_display = min(15, np.percentile(valid_slope, 99))  # Use 99th percentile or 15°

im = ax.imshow(slope_degrees, cmap=slope_cmap,
               extent=extent, origin='upper',
               vmin=0, vmax=vmax_display,
               interpolation='bilinear', zorder=2)

uk_boundary.boundary.plot(ax=ax, color='black', linewidth=1.5, zorder=10)

ax.set_xlabel('Easting (m)', fontsize=14, fontweight='bold')
ax.set_ylabel('Northing (m)', fontsize=14, fontweight='bold')
ax.set_title('Terrain Slope (Degrees)\n(Wind Farm Development Constraint)',
             fontsize=18, fontweight='bold', pad=20)
ax.ticklabel_format(style='plain')
ax.tick_params(labelsize=11)

cbar = plt.colorbar(im, ax=ax, pad=0.02, fraction=0.046, shrink=0.8)
cbar.set_label('Slope (degrees)', fontsize=13, fontweight='bold', labelpad=10)
cbar.ax.tick_params(labelsize=10)

# Add threshold lines
cbar.ax.axhline(y=1, color='lime', linestyle='--', linewidth=1.5, alpha=0.7)
cbar.ax.text(1.35, 1, '1°', fontsize=8, va='center')
cbar.ax.axhline(y=6, color='red', linestyle='--', linewidth=1.5, alpha=0.7)
cbar.ax.text(1.35, 6, '6°', fontsize=8, va='center')

# FIXED: Legend categories that match the actual data range and sum to 100%
legend_elements = [
    mpatches.Patch(color='#ffffff', 
                   label=f'Flat (0-1°): {((valid_slope <= 1).sum()/len(valid_slope)*100):.1f}%', 
                   edgecolor='gray'),
    mpatches.Patch(color='#ffeda0', 
                   label=f'Gentle (1-2°): {(((valid_slope > 1) & (valid_slope <= 2)).sum()/len(valid_slope)*100):.1f}%'),
    mpatches.Patch(color='#feb24c', 
                   label=f'Moderate (2-3°): {(((valid_slope > 2) & (valid_slope <= 3)).sum()/len(valid_slope)*100):.1f}%'),
    mpatches.Patch(color='#fd8d3c', 
                   label=f'Moderately Steep (3-6°): {(((valid_slope > 3) & (valid_slope <= 6)).sum()/len(valid_slope)*100):.1f}%'),
    mpatches.Patch(color='#e31a1c', 
                   label=f'Steep (6-10°): {(((valid_slope > 6) & (valid_slope <= 10)).sum()/len(valid_slope)*100):.1f}%'),
    mpatches.Patch(color='#bd0026', 
                   label=f'Very Steep (>10°): {((valid_slope > 10).sum()/len(valid_slope)*100):.1f}%'),
]

legend = ax.legend(handles=legend_elements, loc='upper left',
                   fontsize=9, framealpha=0.95, edgecolor='black',
                   title='Slope Classification', title_fontsize=10,
                   fancybox=True, shadow=True)

# Verify percentages sum to 100%
total_pct = sum([
    ((valid_slope <= 1).sum()/len(valid_slope)*100),
    (((valid_slope > 1) & (valid_slope <= 2)).sum()/len(valid_slope)*100),
    (((valid_slope > 2) & (valid_slope <= 3)).sum()/len(valid_slope)*100),
    (((valid_slope > 3) & (valid_slope <= 6)).sum()/len(valid_slope)*100),
    (((valid_slope > 6) & (valid_slope <= 10)).sum()/len(valid_slope)*100),
    ((valid_slope > 10).sum()/len(valid_slope)*100)
])
print(f"Total percentage check: {total_pct:.1f}% (should be ~100%)")

stats_text = f"""Slope Statistics (deg):
Mean:   {valid_slope.mean():.2f}
Median: {np.median(valid_slope):.2f}
Std:    {valid_slope.std():.2f}
Max:    {valid_slope.max():.2f}

Coverage by slope:
≤1°:   {((valid_slope <= 1).sum()/len(valid_slope)*100):.1f}%
≤3°:   {((valid_slope <= 3).sum()/len(valid_slope)*100):.1f}%
≤6°:   {((valid_slope <= 6).sum()/len(valid_slope)*100):.1f}%
>6°:   {((valid_slope > 6).sum()/len(valid_slope)*100):.1f}%

Thresholds:
≤1°:  Excellent
1-6°: Declining linearly
>6°:  Unsuitable

Grid: {rows}x{cols}
Valid: {len(valid_slope):,} cells"""

props = dict(boxstyle='round', facecolor='white', alpha=0.95,
             edgecolor='black', linewidth=1.5)
ax.text(0.02, 0.15, stats_text, transform=ax.transAxes,
        fontsize=9, verticalalignment='bottom', bbox=props,
        fontfamily='monospace')

scale_x = extent[0] + (extent[1] - extent[0]) * 0.05
scale_y = extent[2] + (extent[3] - extent[2]) * 0.05
scale_length = 100000

ax.plot([scale_x, scale_x + scale_length], [scale_y, scale_y], 'k-', linewidth=3, zorder=15)
ax.plot([scale_x, scale_x], [scale_y - 5000, scale_y + 5000], 'k-', linewidth=3, zorder=15)
ax.plot([scale_x + scale_length, scale_x + scale_length],
        [scale_y - 5000, scale_y + 5000], 'k-', linewidth=3, zorder=15)
ax.text(scale_x + scale_length/2, scale_y + 12000, '100 km',
        ha='center', fontsize=11, fontweight='bold',
        bbox=dict(boxstyle='round', facecolor='white', alpha=0.9), zorder=15)

arrow_x = extent[1] - (extent[1] - extent[0]) * 0.05
arrow_y = extent[3] - (extent[3] - extent[2]) * 0.08
ax.annotate('N', xy=(arrow_x, arrow_y + 40000), xytext=(arrow_x, arrow_y),
            arrowprops=dict(arrowstyle='->', lw=3, color='black'),
            fontsize=16, fontweight='bold', ha='center', zorder=15)

plt.tight_layout()
plt.savefig(MAP1_FILE, dpi=300, bbox_inches='tight', facecolor='white')
print(f"✓ Saved: {MAP1_FILE}")
plt.close()

# Map 2: Suitability Score
print("\n" + "="*80)
print("MAP 2: Slope Suitability Score")
print("="*80)

fig, ax = plt.subplots(figsize=(14, 18), dpi=300)
ax.set_facecolor('#e6f2ff')
ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5, color='gray', zorder=1)

suit_colors = [
    '#a50026', '#d73027', '#f46d43', '#fdae61', '#fee08b',
    '#ffffbf', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850', '#006837'
]
suit_cmap = LinearSegmentedColormap.from_list('suitability_slope', suit_colors, N=256)

im = ax.imshow(suitability_score, cmap=suit_cmap,
               extent=extent, origin='upper',
               vmin=0, vmax=10,
               interpolation='bilinear', zorder=2)

uk_boundary.boundary.plot(ax=ax, color='black', linewidth=1.5, zorder=10)

ax.set_xlabel('Easting (m)', fontsize=14, fontweight='bold')
ax.set_ylabel('Northing (m)', fontsize=14, fontweight='bold')
ax.set_title('Slope Suitability Score (0-10)\n(Based on Terrain Gradient)',
             fontsize=18, fontweight='bold', pad=20)
ax.ticklabel_format(style='plain')
ax.tick_params(labelsize=11)

cbar = plt.colorbar(im, ax=ax, pad=0.02, fraction=0.046, shrink=0.8)
cbar.set_label('Suitability Score', fontsize=13, fontweight='bold', labelpad=10)
cbar.ax.tick_params(labelsize=10)

# Add score threshold lines
for score in [5, 7, 9]:
    cbar.ax.axhline(y=score, color='gray', linestyle=':', linewidth=1.5, alpha=0.5)

# FIXED: Legend categories based on actual score distribution
legend_elements = [
    mpatches.Patch(color='#006837', 
                   label=f'Excellent (9-10): ≤1° - {((valid_scores >= 9).sum()/len(valid_scores)*100):.1f}%'),
    mpatches.Patch(color='#66bd63', 
                   label=f'Good (7-9): 1-2° - {(((valid_scores >= 7) & (valid_scores < 9)).sum()/len(valid_scores)*100):.1f}%'),
    mpatches.Patch(color='#d9ef8b', 
                   label=f'Fair (5-7): 2-3.5° - {(((valid_scores >= 5) & (valid_scores < 7)).sum()/len(valid_scores)*100):.1f}%'),
    mpatches.Patch(color='#fdae61', 
                   label=f'Marginal (2-5): 3.5-5.5° - {(((valid_scores >= 2) & (valid_scores < 5)).sum()/len(valid_scores)*100):.1f}%'),
    mpatches.Patch(color='#d73027', 
                   label=f'Poor (0-2): >5.5° - {(((valid_scores >= 0) & (valid_scores < 2)).sum()/len(valid_scores)*100):.1f}%'),
]

legend = ax.legend(handles=legend_elements, loc='upper left',
                   fontsize=9, framealpha=0.95, edgecolor='black',
                   title='Suitability Classes', title_fontsize=10,
                   fancybox=True, shadow=True)

stats_text = f"""Score Statistics:
Mean:   {valid_scores.mean():.2f}
Median: {np.median(valid_scores):.2f}
Std:    {valid_scores.std():.2f}
Min:    {valid_scores.min():.2f}
Max:    {valid_scores.max():.2f}

Scoring Formula:
S ≤ 1°:      Score = 10
1° < S ≤ 6°: Score = 10 - 2(S-1)
S > 6°:      Score = 0

Distribution:
Excellent: {((valid_scores >= 9).sum()/len(valid_scores)*100):5.1f}%
Good:      {(((valid_scores >= 7) & (valid_scores < 9)).sum()/len(valid_scores)*100):5.1f}%
Fair:      {(((valid_scores >= 5) & (valid_scores < 7)).sum()/len(valid_scores)*100):5.1f}%
Marginal:  {(((valid_scores >= 2) & (valid_scores < 5)).sum()/len(valid_scores)*100):5.1f}%
Poor:      {((valid_scores < 2).sum()/len(valid_scores)*100):5.1f}%

WLC Weight: 6.5%
Grid: {rows}x{cols}
Valid: {len(valid_scores):,} cells"""

props = dict(boxstyle='round', facecolor='white', alpha=0.95,
             edgecolor='black', linewidth=1.5)
ax.text(0.02, 0.15, stats_text, transform=ax.transAxes,
        fontsize=8.5, verticalalignment='bottom', bbox=props,
        fontfamily='monospace')

ax.plot([scale_x, scale_x + scale_length], [scale_y, scale_y], 'k-', linewidth=3, zorder=15)
ax.plot([scale_x, scale_x], [scale_y - 5000, scale_y + 5000], 'k-', linewidth=3, zorder=15)
ax.plot([scale_x + scale_length, scale_x + scale_length],
        [scale_y - 5000, scale_y + 5000], 'k-', linewidth=3, zorder=15)
ax.text(scale_x + scale_length/2, scale_y + 12000, '100 km',
        ha='center', fontsize=11, fontweight='bold',
        bbox=dict(boxstyle='round', facecolor='white', alpha=0.9), zorder=15)

ax.annotate('N', xy=(arrow_x, arrow_y + 40000), xytext=(arrow_x, arrow_y),
            arrowprops=dict(arrowstyle='->', lw=3, color='black'),
            fontsize=16, fontweight='bold', ha='center', zorder=15)

plt.tight_layout()
plt.savefig(MAP2_FILE, dpi=300, bbox_inches='tight', facecolor='white')
print(f"✓ Saved: {MAP2_FILE}")
plt.close()

print("\n" + "="*80)
print("VISUALIZATION COMPLETE")
print("="*80)
print(f"\nKey fixes:")
print(f"  1. Legend categories now cover full data range (0-10+°)")
print(f"  2. Percentages sum to 100%: {total_pct:.1f}%")
print(f"  3. Colorbar range adjusted to show actual data distribution")
print(f"  4. WLC weight updated to 6.5%")
print(f"\nOutput files:")
print(f"  {MAP1_FILE}")
print(f"  {MAP2_FILE}")
print("="*80)

SLOPE VISUALIZATION (FIXED)

Loading reference grid
Shape: 235 x 131
Extent: X[0, 655000], Y[10000, 1185000]

Loading raster data
Loading UK boundary
Valid slope cells: 10,397
Valid score cells: 10,397

Slope statistics:
  Min: 0.00°
  Max: 8.79°
  Mean: 1.19°
  Median: 0.82°

Slope distribution:
  ≤ 1°:   5,907 cells ( 56.8%)
  ≤ 2°:   8,399 cells ( 80.8%)
  ≤ 3°:   9,508 cells ( 91.4%)
  ≤ 6°:  10,370 cells ( 99.7%)
  ≤10°:  10,397 cells (100.0%)
  ≤15°:  10,397 cells (100.0%)

MAP 1: Slope (Degrees)


C:\Users\888\AppData\Local\Temp\ipykernel_31336\1025099571.py:119: UserWarning: Setting the 'color' property will override the edgecolor or facecolor properties.
  mpatches.Patch(color='#ffffff',


Total percentage check: 100.0% (should be ~100%)
✓ Saved: D:\BENV0093\outputs\map7_slope_degrees.png

MAP 2: Slope Suitability Score
✓ Saved: D:\BENV0093\outputs\map8_slope_suitability_score.png

VISUALIZATION COMPLETE

Key fixes:
  1. Legend categories now cover full data range (0-10+°)
  2. Percentages sum to 100%: 100.0%
  3. Colorbar range adjusted to show actual data distribution
  4. WLC weight updated to 6.5%

Output files:
  D:\BENV0093\outputs\map7_slope_degrees.png
  D:\BENV0093\outputs\map8_slope_suitability_score.png
